<a href="https://colab.research.google.com/github/Datarian723/AutoHPOJupyter/blob/main/AutoHpoFINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODO

In [18]:
"""
MIGRATION OF objective FROM: Objective TO: ConfigSpace IN PROGRESS.
(5)MOVE_TO: KaggleMTCSVariants.ipnyb. TEST: AutoHPO.(Now you can put in other 'action_funcs' as well; 'dls_action', 'dset_action'...etc)
(6)WHILE_RUNNING_(5): START: https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/data


(USER_GUIDE)GUIDE: IF (the number of action_func(s)) << (the number of params),
                   THEN recommend the user to create original_func, original_func_var1, original_func_var2,...etc
                   (where original_func_var(i) is simply an i'th COPY of the original_func. This works!!!!!)
"""

"\nMIGRATION OF objective FROM: Objective TO: ConfigSpace IN PROGRESS.\n(5)MOVE_TO: KaggleMTCSVariants.ipnyb. TEST: AutoHPO.(Now you can put in other 'action_funcs' as well; 'dls_action', 'dset_action'...etc)\n(6)WHILE_RUNNING_(5): START: https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/data\n\n\n(USER_GUIDE)GUIDE: IF (the number of action_func(s)) << (the number of params),\n                   THEN recommend the user to create original_func, original_func_var1, original_func_var2,...etc\n                   (where original_func_var(i) is simply an i'th COPY of the original_func. This works!!!!!)\n"

In [19]:
!pip install fastai --upgrade --quiet
!pip install optuna --upgrade --quiet
!pip install optuna-integration --upgrade --quiet
!pip install scipy --upgrade --quiet

#Prep

In [20]:
# ---------------------------
# Special Imports
# ---------------------------
from __future__ import annotations  # Enable forward references for type hints

# ---------------------------
# Standard Libraries
# ---------------------------
import os  # Operating system utilities
import gc  # Garbage collection for memory management
import logging  # Logging utilities
import sys  # System-specific parameters and functions
import unittest  # For unit-testing Python code
import copy  # For deep and shallow copying of objects
import math  # Math operations
import random  # Random number generation
import gzip  # For compressing and decompressing data
import ast #For building an Abstract Syntax Tree; used in class 'Objective'.
import warnings  # For handling warnings
import textwrap #For dedenting texts.
from functools import partial  # Partial function application
from collections import defaultdict  # Dictionary subclass with default values
from itertools import chain # Used for 'flattening' Collection of Collections.
from sortedcontainers import SortedSet  # Sorted set implementation
from unittest.mock import Mock, patch  # For mocking and patching in unit tests
from inspect import signature  # Inspecting callable signatures
from typing import Callable, Union  # Type annotations for type hints
from ast import Is  # Abstract syntax tree utilities
from abc import ABC, abstractmethod  # Abstract Base Classes for inheritance

# ---------------------------
# Scientific Libraries
# ---------------------------
import numpy as np  # Array and matrix operations
import pandas as pd  # Data manipulation and analysis
import matplotlib.pyplot as plt  # Plotting and visualization

# ---------------------------
# PyTorch and Related Modules
# ---------------------------
import torch  # Core PyTorch library for tensor operations
import torch.nn as nn  # Neural network modules
import torch.nn.functional as F  # Functional API for neural networks
import torchvision.models as models  # Pre-trained models from torchvision
from torch.multiprocessing import Pool, set_start_method  # Multiprocessing utilities for PyTorch

# ---------------------------
# FastAI Libraries
# ---------------------------
from fastai.vision.all import *  # All-in-one import for vision-specific modules
from fastai.metrics import *  # Evaluation metrics for model performance
from fastai.callback.hook import *  # Hooks for capturing intermediate model states
from fastai.callback.tracker import *  # Training callback trackers (e.g., early stopping)
from fastai.learner import Learner  # Core Learner object for model training

# ---------------------------
# Optuna for Hyperparameter Optimization
# ---------------------------
import optuna  # Core Optuna library for optimization
from optuna.integration import FastAIPruningCallback  # Pruning callback for FastAI integration
from optuna.trial import Trial, FrozenTrial  # Trial classes for defining and managing trials

# ---------------------------
# Google Colab Utilities
# ---------------------------
from google.colab import files  # File utilities for Colab
from google.colab import drive  # Mounting Google Drive in Colab

#Class: Var

In [21]:
class Var:
    def __init__(self, var_type: str, source: dict, belongs_to: str=None):
        sample_method = source['sample']
        distribution = source['params']['choices'] if sample_method=='categorical' else source['params']
        #ERROR CHECK:
        self._validate_inputs(distribution, sample_method)

        #Member initializations:
        self.var_type = var_type
        self.sample_method = sample_method
        self._belongs_to = belongs_to
        #frozen_dist is used for value conversion, in case value NOT currently in 'self.get_dist()' is requested for conversion.
        self._frozen_dist = distribution.copy()
        self._dist = list(range(len(self._frozen_dist))) if sample_method=='categorical' else distribution.copy()

    def _validate_inputs(self, distribution, sample_method: str):
        if not isinstance(distribution, (list,dict)):
            raise ValueError(f"Input 'distribution' MUST be of type in (list,dict) but found: {type(distribution)}")
        if sample_method not in ['categorical', 'float', 'int']:
            raise ValueError(f"sample method MUST be one of ['categorical', 'float', 'int'] but found: {sample_method}")
        if sample_method!='categorical' and not isinstance(distribution, dict) or sample_method=='categorical' and not isinstance(distribution, list):
            raise ValueError(f"sample_method: {sample_method} is not compatible with distribution: {distribution}. Categorical sampling requires a list, rest requires a dictionary")
        cat_valid_params = sample_method!='categorical' or len(distribution)
        non_cat_valid_params = sample_method=='categorical' or (distribution.get('low', 0) <= distribution.get('high', -1))
        if not cat_valid_params or not non_cat_valid_params:
            msg = f"Categorical distribution needs at least one value in it: {distribution}" if sample_method=='categorical' else f"distribution needs (low,high) as keys and distribution[low]<=distribution[high] but found: {distribution}"
            raise ValueError(msg)

    def __hash__(self):
        return hash((self.var_type, self._belongs_to))

    def __eq__(self, other: Var):
        return all([
            isinstance(other, Var),
            self.var_type == other.var_type,
            self.sample_method == other.sample_method,
            self._belongs_to == other._belongs_to,
            self._frozen_dist == other._frozen_dist,
            self._dist == other._dist
        ])

    def __str__(self):
        if self.sample_method == 'categorical':
            config_space = f"'choices' = {[self._frozen_dist[idx] for idx in self._dist]}"
        else:
            config_space = (
                f"'low': {self._dist['low']}, "
                f"'high': {self._dist['high']}, "
                f"'log': {self._dist.get('log', False)}, "
                f"'step': {self._dist.get('step', 'unspecified')}"
            )
        output = f"Var of type: {self.var_type}, signature of: {self._belongs_to}\nConfiguration Space:\n{config_space}\n"
        return output

    def is_fixed(self):
        return len(self._dist)==1 if self.sample_method=='categorical' else self._dist['low']==self._dist['high']

    def is_sig_var(self):
        return self._belongs_to!='None'

    def belongs_to(self):
        return self._belongs_to

    def contain_idx(self, idx, current: bool = False):
        if self.sample_method!='categorical' and type(idx)!=eval(self.sample_method) or self.sample_method=='categorical' and type(idx)!=int:
            return False
        if self.sample_method!='categorical':
            dist = self._dist if current else self._frozen_dist
            return dist['low'] <= idx <= dist['high']
        return idx in self._dist if current else 0 <= idx < len(self._frozen_dist)

    def contain_val(self, val, current: bool = False):
        if self.sample_method != 'categorical':
            return self.contain_idx(idx=val, current=current)
        if self.var_type=='action_func' and isinstance(val, Action):
            val = val.action_func
        return val in self.distribution() if current else val in self._frozen_dist

    def convert_idx_to_val(self, idx):
        if not self.contain_idx(idx=idx, current=False):
            raise KeyError(f"Input index '{idx}' is NOT a proper index for conversion via Var:{self.var_type}'s distribution: {list(range(len(self._frozen_dist))) if self.sample_method=='categorical' else self._frozen_dist}")
        if self.sample_method!='categorical':
            return idx
        return self._frozen_dist[idx]

    def convert_val_to_idx(self, val):
        if not self.contain_val(val=val, current=False):
            raise ValueError(f"Input value '{val}' is NOT a proper value for conversion via Var:{self.var_type}'s distribution: {self._frozen_dist}")
        if self.sample_method!='categorical':
            return val
        return self._frozen_dist.index(val)

    def distribution(self, indices: bool = False):
        if self.sample_method!='categorical':
            return self._dist
        if indices:
            return self._dist
        return [self._frozen_dist[idx] for idx in self._dist]

    def sample(self, trial):
        if not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' must be of type in (optuna::Trial, optuna::FrozenTrial) but found: {trial}")
        name=self.var_type+'>'+str(self._belongs_to)
        if isinstance(trial, FrozenTrial):
            idx_or_val = trial.params[name]
            #sample_method=='categorical' <-> retrieved value is an index NOT an actual value.
            sampled_val = self.convert_idx_to_val(idx_or_val) if self.sample_method=='categorical' else idx_or_val
            return sampled_val

        sampled_val = None
        if self.sample_method == 'categorical':
            idx = trial.suggest_categorical(name, choices=self._dist)
            sampled_val = self._frozen_dist[idx]
        elif self.sample_method == 'float':
            sampled_val = trial.suggest_float(name, **self._dist)
        elif self.sample_method == 'int':
            sampled_val = trial.suggest_int(name, **self._dist)

        return sampled_val

    def update(self, vals: list):
        #Proior processing:
        if not len(vals):
            return
        if not all(isinstance(val, (int,float)) for val in vals):
            raise ValueError(f"Var of type: {self.var_type} with sample_method: {self.sample_method} recieved update request with 'vals': {vals} which contain element(s) NOT of type in (int,float)")
        vals = sorted(vals.copy()) if self.sample_method=='categorical' else vals

        set_vals = set(vals)
        if len(set_vals)==1:
            vals = [vals[0]]

        #ERROR CHECKS:
        if self.sample_method!='categorical' and (min(vals)<self._dist['low'] or max(vals)>self._dist['high']):
            raise ValueError(f"Var of type: {self.var_type} with sample_method: {self.sample_method} recieved update request with 'vals': {vals} NOT in valid range: [{self._dist['low']},{self._dist['high']}]. Note the initial search space: [{self._frozen_dist['low']},{self._frozen_dist['high']}].")
        if self.sample_method=='categorical' and not set_vals.issubset(set(self._dist)):
            raise ValueError(f"Var of type: {self.var_type} with sample_method: {self.sample_method} recieved update request with 'vals': {vals} NOT in valid range: {self._dist}. Note the initial search space: {self._frozen_dist}")

        #UPDATE: configuration space.
        if self.sample_method=='categorical':
            self._dist = vals
        else:
            self._dist['low'] = min(vals)
            self._dist['high'] = max(vals)

#Test: Var

In [ ]:
class TestVar(unittest.TestCase):

    def setUp(self):
        self.categorical_source = {
            'params': {'choices': [1, 2, 3, 4, 5]},
            'sample': 'categorical'
        }
        self.float_source = {
            'params': {'low': 0.1, 'high': 1.0},
            'sample': 'float'
        }
        self.int_source = {
            'params': {'low': 1, 'high': 10},
            'sample': 'int'
        }

    def test_init_categorical(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.var_type, 'test_var')
        self.assertEqual(var.sample_method, 'categorical')
        self.assertEqual(var._frozen_dist, [1, 2, 3, 4, 5])
        self.assertEqual(var._dist, [0, 1, 2, 3, 4])
        self.assertFalse(var.is_fixed())

    def test_init_float(self):
        var = Var(var_type='test_var', source=self.float_source)
        self.assertEqual(var.var_type, 'test_var')
        self.assertEqual(var.sample_method, 'float')
        self.assertEqual(var._frozen_dist, {'low': 0.1, 'high': 1.0})
        self.assertEqual(var._dist, {'low': 0.1, 'high': 1.0})
        self.assertFalse(var.is_fixed())

    def test_init_int(self):
        var = Var(var_type='test_var', source=self.int_source)
        self.assertEqual(var.var_type, 'test_var')
        self.assertEqual(var.sample_method, 'int')
        self.assertEqual(var._frozen_dist, {'low': 1, 'high': 10})
        self.assertEqual(var._dist, {'low': 1, 'high': 10})
        self.assertFalse(var.is_fixed())

    def test_init_error_invalid_sample_method(self):
        with self.assertRaises(ValueError):
            Var(var_type='test_var', source={'params': {'choices': [1, 2]}, 'sample': 'invalid'})

    def test_init_error_invalid_distribution_type(self):
        with self.assertRaises(ValueError):
            Var(var_type='test_var', source={'params': {'choices': {'key': 'value'}}, 'sample': 'categorical'})

    def test_contain_idx(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertTrue(var.contain_idx(2))
        self.assertFalse(var.contain_idx(5))

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertTrue(var_float.contain_idx(0.5))
        self.assertFalse(var_float.contain_idx(1.1))

    def test_contain_val(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertTrue(var.contain_val(3))
        self.assertFalse(var.contain_val(6))

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertTrue(var_float.contain_val(0.5))
        self.assertFalse(var_float.contain_val(1.1))

    def test_convert_idx_to_val(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.convert_idx_to_val(2), 3)

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertEqual(var_float.convert_idx_to_val(0.5), 0.5)

    def test_convert_val_to_idx(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.convert_val_to_idx(3), 2)

        with self.assertRaises(ValueError):
            var.convert_val_to_idx(6)

    def test_distribution(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.distribution(), [1, 2, 3, 4, 5])

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertEqual(var_float.distribution(), {'low': 0.1, 'high': 1.0})

    def test_sample_categorical(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        trial = optuna.create_study(direction='minimize').ask()
        trial.suggest_categorical = Mock(return_value=2)
        sampled_val = var.sample(trial)
        self.assertEqual(sampled_val, 3)

    def test_sample_float(self):
        var = Var(var_type='test_var', source=self.float_source)
        trial = optuna.create_study(direction='minimize').ask()
        trial.suggest_float = Mock(return_value=0.5)
        sampled_val = var.sample(trial)
        self.assertEqual(sampled_val, 0.5)

    def test_sample_error(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        with self.assertRaises(ValueError):
            var.sample('not_a_trial')

    def test_update_categorical(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        var.update([1, 2])
        self.assertEqual(var.distribution(indices=True), [1, 2])
        self.assertEqual(var.distribution(), [2, 3])
        self.assertFalse(var.is_fixed())

    def test_update_float(self):
        var = Var(var_type='test_var', source=self.float_source)
        var.update([0.2, 0.9])
        self.assertEqual(var._dist['low'], 0.2)
        self.assertEqual(var._dist['high'], 0.9)

    def test_update_error_out_of_range(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        with self.assertRaises(ValueError):
            var.update([0, 6])

    def test_str(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        expected_str = f"Var of type: test_var, signature of: {'None'}\nConfiguration Space:\n'choices' = [1, 2, 3, 4, 5]\n"
        self.assertEqual(str(var), expected_str)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

#Class: Action, ModelAction

In [22]:
class Action:
    def __init__(self, action_func: Callable, sig_dict: dict):
        if not all(isinstance(var,Var) for var in sig_dict.values()):
            raise ValueError(f"Constructor of object 'Action' with action_func: '{action_func.__name__}' received an invalid dictionary sig_dict: {sig_dict}")
        self.action_func = action_func
        self.sig_dict = sig_dict
        self.sig = frozenset(sig_dict.keys())
        self.params = {}
        self.action_state = None
        self._model_loader = None

    def __len__(self):
        return len(self.sig_dict)

    def __hash__(self):
        return hash(self.action_func)

    def __eq__(self, other: Action):
        """Two Action objects are considered 'equal'
        IFF (1)other is Action object
            (2)self.action_func == other.action_func
            (3)self.params == other.params
            (4)self.action_state == other.action_state
        """
        #TYPE CHECK:
        if (isinstance(self, ModelAction) and not isinstance(other, ModelAction)) or not isinstance(other, Action):
            return False
        #Model loader comparison:
        self_model_params = self._model_loader if self._model_loader is None else self._model_loader.keywords
        other_model_params = other._model_loader if other._model_loader is None else other._model_loader.keywords
        if self_model_params != other_model_params:
            return False

        #Compare action states if both are dictionaries
        if isinstance(self.action_state, dict) and isinstance(other.action_state, dict):
            if self.action_state.keys() != other.action_state.keys():
                return False
            if not all(torch.equal(self.action_state[key], other.action_state[key]) for key in self.action_state.keys()):
                return False
        elif self.action_state != other.action_state: #IF one is None and the other is not None.
            return False

        return all([
            self.action_func == other.action_func,
            self.sig_dict == other.sig_dict,
        ])

    def __str__(self):
        if self._model_loader:
            parameters = " ".join([f"{key}={val}" for key,val in self._model_loader.keywords.items()])
        else:
            parameters = " ".join([f"{key}" for key in self.sig_dict.keys()])
        return f"Action of func: {self.action_func.__name__}\n" + f"Model loader parameters: {parameters}\n"

    def get_var(self, var_type):
        if var_type not in self.keys():
            raise KeyError(f"'{var_type}' is not a valid signature key for Action: '{self.action_func.__name__}'. Available keys: {set(self.keys())}")
        return self.sig_dict[var_type]

    def keys(self):
        return self.sig_dict.keys()

    def values(self):
        return self.sig_dict.values()

    def items(self):
        return self.sig_dict.items()

    def is_loadable(self):
        return self._model_loader is not None

    def is_fixed(self):
        return all(var.fixed for var in self.sig_dict.values())

    def parameterize(self, trial: optuna.Trial):
        """
        ACCEPTS:
        trial := Active optuna trial objet.
                 IF input 'manual_action_params' is not empty, THEN 'trial' will NOT be used.
        RETURNS: void.
                 The function parameterizes the caller Action object.
                 Without any saved action_state, the caller Action object needs to be parameterized every time before .__call__()
        """
        #ERROR CHECKS:
        if self.action_state is not None and not self.params:
            raise RuntimeError(f"Action: {self.action_func.__name__} has a saved action state(self.action_state!=None) but NO saved params(self.params==None).")
        if self.action_state is None and trial is None:
            raise ValueError(f"Action: {self.action_func.__name__} has no saved action state(self.action_state==None) and no input trial passed in(self.trial==None). No sampling can be done.")

        if self.action_state is None:
            self.params = {cat:var.sample(trial=trial) for cat,var in self.items()}

        #self._model_loader <- Fully parameterized model with self.params
        #NOTE: IF it has a saved action state, the consistent self.params will restore the recenmost state(based on the recentmost top sampling results)
        self._model_loader = partial(self.action_func, **self.params)

    def __call__(self, device: torch.device):
        if self.action_state is not None and not self.params:
            raise RuntimeError(f"Action: {self.action_func.__name__} has a saved action state(self.action_state!=None) but no saved params exists(self.params empty). Likely an internal logic issue")
        if self._model_loader is None:
            raise RuntimeError(f"Action: {self.action_func.__name__} cannot be called. Call Action.parameterize() first and then try it again.")

        #Try: Applying 'device' to the model_loader and load.
        try:
            model = self._model_loader(device=device)
        except Exception as e: #IF fails: Load the model w/o device.
            model = self._model_loader()

        #IF 'self.action_state' exists, THEN load the saved weights from the state dictionary(=self.action_state) to the model
        if self.action_state:
            try:
                name = self.action_func.__name__
                model.load_state_dict(self.action_state)
            except torch.nn.modules.module.ModuleAttributeError as e:
                raise ValueError(f"Attribute error: {e}. The state dictionary of Action: '{name}' may not match the model.")
            except RuntimeError as e:
                raise RuntimeError(f"Runtime error: {e}. The state dictionary of Action: '{name}' may have unmatching layer sizes or missing keys.")
            except Exception as e:
                raise ValueError(f"An unexpected error occurred while loading state dictionary to Action '{name}': {e}")

        #Last but not least, return the model.
        return model

    def update_sig_vars(self, update_dict: dict):
        #IF Action (1)has a saved 'action_state' OR (2)0 elements in the 'list_params'
        #THEN NO update shall be done.
        if self.action_state or not len(update_dict):
            return

        leftovers = set(self.keys()) - set(update_dict.keys())
        if leftovers:
            raise RuntimeError(f"Action: {self.action_func.__name__} requires following missing keys in 'update_dict': {leftovers}")

        #Update: signature configuration space(=='self.sig_dict')
        for var_type,var in self.items():
            var.update(vals=update_dict[var_type])

    #Calling Action.update_state() will raise a helpful Error.
    def update_state(self, obj):
        return NotImplementedError("Erroneous call to Action.update_state()")

class ModelAction(Action):
    def __init__(self, action_func: Callable, sig_dict: dict):
        super().__init__(action_func=action_func, sig_dict=sig_dict)

    def update_state(self, obj):
        #Ensure that input obj has a correct type/format.
        if not(isinstance(obj, (nn.Module, dict)) or hasattr(obj, 'model')):
            raise ValueError(f"Updating action state of Action: {self.action_func.__name__} failed. Input object 'obj' must either be of type in (nn.Module, dict) or must contain 'model' as an attribute but none of the two conditions are satisfied.")
        #Ensure that if obj.model exists, then it points to an instance of nn.Module.
        if hasattr(obj, 'model') and not isinstance(obj.model, nn.Module):
            raise ValueError(f"Input obj.model must be of type nn.Module but found type(obj.model): {type(obj.model)}")
        #Ensure that no update is done if the model was not sampled in the previous run.
        if self.action_state is not None and self.params != self._model_loader.keywords:
            print(f"ModelAction: {self.action_func.__name__} was not sampled in the previous HPO run or no HPO run has been executed yet. This call will not change the action state")
            return

        if isinstance(obj, dict):
            self.action_state = obj.copy() #shallow copy it for safety
        else:
            self.action_state = obj.state_dict() if isinstance(obj, nn.Module) else obj.model.state_dict()

        #Manually set 'self.params'. This way, saved action_state -> self.params available!
        self.params = self._model_loader.keywords.copy()

#Test: Action, ModelAction

In [ ]:
"""
Parameters used in the testin of class 'ModelAction' defined in this cell
"""
if __name__ == '__main__':
    class ModelDefault(nn.Module):
        def __init__(self, rand_number: int):
            super().__init__()
            self.net = nn.Linear(16,10)
            self.rand_number = torch.tensor(rand_number)

        def forward(self, x):
            return self.net(x) + self.rand_number

    class ModelDropout(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: int):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(in_features=16,out_features=10),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=10,out_features=10)
            )

        def forward(self, x):
            return self.net(x)

    def action_default(rand_number: int, device: torch.device):
        return ModelDefault(rand_number=rand_number).to(device)

    def action_dropout(activation, dropout_p, device: torch.device):
        #model_dropout := Class header of 'ModelDropout'
        return ModelDropout(activation=activation, dropout_p=dropout_p).to(device)

    #ASSUMED: Input size of 10x16. (Single batch == 10 (4x4 flattened vector)s )
    #FIRST_LAYER: Fully connected linear layer of size 16x10
    #ACTIVATION: One of nn.ReLU, nn.PReLU, nn.SiLU
    #DROPOUT: nn.Dropout(p=dropout_p)
    #OUT: .shape == [10,10]
    #NOTE: Input 'model: Callable' will be the class header 'Model'.

In [ ]:
class TestModelAction(unittest.TestCase):
    def setUp(self):
        default_sig_dict = {
            'rand_number': Var(var_type='rand_number', source={'params': {'choices': list(range(1, 11))}, 'sample': 'categorical'})
        }
        dropout_sig_dict = {
            'activation': Var(var_type='activation', source={'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]}, 'sample': 'categorical'}),
            'dropout_p': Var(var_type='dropout_p', source={'params': {'low': 0.0, 'high': 1.0}, 'sample': 'float'})
        }

        self.action_sampler = {
            action_default: ModelAction(action_default, default_sig_dict),
            action_dropout: ModelAction(action_dropout, dropout_sig_dict)
        }

        #Create a trial object for parameterization
        self.trial = optuna.create_study(direction='minimize').ask()

    def test_initialization(self):
        default_action = self.action_sampler[action_default]
        dropout_action = self.action_sampler[action_dropout]

        #Test if ModelAction objects are initialized correctly
        self.assertIsInstance(default_action, ModelAction)
        self.assertIsInstance(dropout_action, ModelAction)

        self.assertTrue(default_action.action_func == action_default)
        self.assertIsInstance(default_action.get_var('rand_number'), Var)
        self.assertTrue(
            default_action.get_var('rand_number').distribution() == list(range(1, 11)),
            msg=f"{default_action.get_var('rand_number').distribution()} != {list(range(1, 11))}"
        )
        self.assertFalse(default_action.params)
        self.assertIsNone(default_action.action_state)
        self.assertIsNone(default_action._model_loader)

        self.assertTrue(dropout_action.action_func == action_dropout)
        self.assertIsInstance(dropout_action.get_var('activation'), Var)
        self.assertIsInstance(dropout_action.get_var('dropout_p'), Var)
        self.assertTrue(dropout_action.get_var('activation').distribution() == [nn.ReLU, nn.PReLU, nn.SiLU])
        self.assertFalse(dropout_action.params)
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNone(dropout_action._model_loader)

    def test_default_parameterize_with_action_state(self):
        default_action = self.action_sampler[action_default]
        default_action.action_state = 'dummy'
        default_action.params = {'rand_number': 1}

        #Test parameterization with the default action
        default_action.parameterize(trial=self.trial)

        #Posterior testing
        self.assertIsInstance(default_action, ModelAction)
        self.assertIsNotNone(default_action.params)
        self.assertIsNotNone(default_action.action_state)
        self.assertIsNotNone(default_action._model_loader)

        try:
            model = default_action(device=torch.device('cpu'))
        except Exception as e:
            pass

    def test_default_parameterize(self):
        default_action = self.action_sampler[action_default]
        #Test parameterization with the default action
        default_action.parameterize(trial=self.trial)
        self.assertTrue(set(default_action.params.keys()), set(['rand_number']))
        self.assertTrue(default_action.params['rand_number'] in list(range(1, 11)))
        self.assertIsNone(default_action.action_state)
        self.assertIsNotNone(default_action._model_loader)
        model = default_action(device=torch.device('cpu'))
        X = torch.randn(size=[10, 16])
        Y = model(X)
        self.assertEqual(Y.shape, torch.Size([10, 10]))

        self.assertIsNotNone(default_action.params)
        self.assertIsNone(default_action.action_state)
        self.assertIsNotNone(default_action._model_loader)

    def test_dropout_parameterize_with_action_state(self):
        dropout_action = self.action_sampler[action_dropout]
        dropout_action.params = {
            'activation': nn.ReLU,
            'dropout_p': 0.5
        }
        model = ModelDropout(activation=nn.ReLU, dropout_p=0.5).to(torch.device('cpu'))
        dropout_action.action_state = model.state_dict()

        #Test parameterization with the default action
        dropout_action.parameterize(trial=self.trial)

        #Posterior testing
        self.assertIsInstance(dropout_action, ModelAction)
        self.assertIsNotNone(dropout_action.params)
        self.assertIsNotNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

        try:
            model = dropout_action(device=torch.device('cpu'))
            self.assertTrue(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))
        except Exception as e:
            pass

        #Final testing
        self.assertIsNotNone(dropout_action.params)
        self.assertIsNotNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

    def test_dropout_parameterize(self):
        dropout_action = self.action_sampler[action_dropout]

        #Prior Testing
        self.assertIsInstance(dropout_action, ModelAction)
        self.assertTrue(dropout_action.params == {})
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNone(dropout_action._model_loader)

        #Test parameterization with the default action
        dropout_action.parameterize(trial=self.trial)

        #Posterior Testing
        self.assertTrue(set(dropout_action.params.keys()), set(['activation', 'dropout_p']))
        self.assertTrue(dropout_action.params['activation'] in [nn.ReLU, nn.PReLU, nn.SiLU])
        self.assertTrue(0.0 <= dropout_action.params['dropout_p'] <= 1.0)
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

        #Model loading & testing
        model = dropout_action(device=torch.device('cpu'))
        self.assertEqual(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

        #Final testing
        self.assertIsNotNone(dropout_action.params)
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

    def test_update_sig_vars(self):
        #Preparation
        default_action = self.action_sampler[action_default]
        dropout_action = self.action_sampler[action_dropout]

        default_new_sig_params = {
            'rand_number': [2]
        }
        dropout_new_sig_params = {
            'activation': [1, 2],
            'dropout_p': [0.45, 0.5, 0.72]
        }

        #TEST: default_action FOR: sig_dict
        default_action.update_sig_vars(update_dict=default_new_sig_params)
        self.assertIsInstance(default_action.get_var('rand_number'), Var)
        self.assertTrue(default_action.get_var('rand_number').distribution() == [3])
        self.assertTrue(default_action.get_var('rand_number').distribution(indices=True) == [2])

        #TEST: dropout_action FOR: sig_dict
        dropout_action.update_sig_vars(update_dict=dropout_new_sig_params)
        self.assertIsInstance(dropout_action.get_var('activation'), Var)
        self.assertTrue(dropout_action.get_var('activation').distribution() == [nn.PReLU, nn.SiLU],
                        msg=f"{dropout_action.get_var('activation').distribution()} != {[nn.PReLU, nn.SiLU]}")
        self.assertTrue(dropout_action.get_var('activation').distribution(True) == [1, 2],
                        msg=f"{dropout_action.get_var('activation').distribution(True)} != {[1, 2]}")
        self.assertIsInstance(dropout_action.get_var('dropout_p'), Var)
        dropout_p_var = dropout_action.get_var('dropout_p')
        self.assertTrue(dropout_p_var.distribution()['low'] == 0.45 and dropout_p_var.distribution()['high'] == 0.72)

    #Additional method to test update_state if needed
    def test_update_state(self):
        for i in range(5):
            default_action = self.action_sampler[action_default]
            dropout_action = self.action_sampler[action_dropout]

            if not i:
                self.assertTrue(default_action.params == {})
                self.assertIsNone(default_action.action_state)
                self.assertIsNone(default_action._model_loader)

                self.assertTrue(dropout_action.params == {})
                self.assertIsNone(dropout_action.action_state)
                self.assertIsNone(dropout_action._model_loader)
            else:
                self.assertFalse(default_action.params == {})
                self.assertIsNotNone(default_action.action_state)
                self.assertIsNotNone(default_action._model_loader)

                self.assertFalse(dropout_action.params == {})
                self.assertIsNotNone(dropout_action.action_state)
                self.assertIsNotNone(dropout_action._model_loader)

            default_action.parameterize(trial=self.trial)
            dropout_action.parameterize(trial=self.trial)

            default_model = default_action(device=torch.device('cpu'))
            dropout_model = dropout_action(device=torch.device('cpu'))

            self.assertEqual(default_model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))
            self.assertEqual(dropout_model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

            default_action.update_state(obj=default_model)
            dropout_action.update_state(obj=dropout_model)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

#Class ActionVar

In [23]:
class ActionVar(Var):
    """
    ActionVar is a subclass of Var that stores Action/ModelAction objects in its `_frozen_dist`.
    This enables automated sampling of actions and their parameters during HPO.
    """

    def __init__(self, var_type: str, source: dict, belongs_to: str=None):
        super().__init__(var_type=var_type, source=source, belongs_to=belongs_to)

    def is_fixed(self):
        """
        ActionVar is fixed if:
        - Only one sampleable value in the configuration space.
        - All signature variables of the ActionVar are fixed.
        """
        if len(self._dist) > 1:
            return False
        singleton_action = self._frozen_dist[self._dist[0]]
        return all(var.is_fixed() for var in singleton_action.values())

    def get_action(self, action_idx_or_action_func):
        if isinstance(action_idx_or_action_func, int):
            return self.convert_idx_to_val(action_idx_or_action_func)
        action_list = [action for action in self._frozen_dist if action.action_func == action_idx_or_action_func]
        if not action_list:
            raise KeyError(f"No action found with action_func: {action_idx_or_action_func} in ActionVar: {self.var_type}.")
        return action_list[0]

    def get_var(self, action_idx_or_action_func, var_type):
        action = self.get_action(action_idx_or_action_func)
        return action.get_var(var_type)

    def get_loadable_action(self, action_identifier, source) -> Action:
        """
        Retrieves a fully parameterized Action object.

        Args:
            action_identifier: An instance of Action, a Callable action function, or an integer index.
            source: A dictionary or optuna Trial/FrozenTrial containing parameter mappings.

        Returns:
            A fully parameterized Action object.
        """
        if not isinstance(source, (Trial, FrozenTrial, dict)):
            raise ValueError(f"Input 'source' must be of type dict, Trial, or FrozenTrial, but found: {type(source)}")
        if not (isinstance(action_identifier, (int, Action)) or callable(action_identifier)):
            raise TypeError(f"Input 'action_identifier' should be an int, Action, or Callable, but found: {type(action_identifier)}")

        #Get the action based on the identifier
        if isinstance(action_identifier, Action):
            action = action_identifier
        else:
            action = self.get_action(action_identifier)

        #If the action has a saved state, return it
        if action.action_state:
            return action

        #Extract parameters from the source
        source = source if isinstance(source, dict) else source.params
        params = {
            var_type: val for (var_type, belongs_to), val in source.items()
            if var_type in action.keys() and belongs_to == self.var_type
        }

        #Check for missing parameters
        missing_params = set(action.keys()) - set(params.keys())
        if missing_params:
            raise KeyError(f"Action '{action.action_func.__name__}' requires missing keys in 'source': {missing_params}. \nNOTE: 'source': {source}, 'params': {params}")

        #Convert indices to actual values
        for var_type, var in action.items():
            val = params[var_type]
            if var.contain_idx(val):
                val = var.convert_idx_to_val(val)
            params[var_type] = val

        #Update the action's model loader
        action._model_loader = partial(action.action_func, **params)

        return action

    def update_actions(self, update_action_idxs: list, list_params: list, update_state_window: int, device: torch.device):
        """
        Updates actions based on recent sampling results.

        Args:
            update_action_idxs: List of unique indices representing actions to update.
            list_params: List of parameter dictionaries from recent optuna trials.
                         Every 'params' in the list_params is guaranteed to have (self.var_type,'None') as its key and therefore,
                         will have all the (self.keys(),self.var_type) as keys as well.
            update_state_window: Number of top actions to update states for.
            device: Torch device to use for model actions.
        """
        update_state_window = min(update_state_window, len(update_action_idxs))
        has_meaningful_state = isinstance(self.distribution()[0], ModelAction)
        for count,idx in enumerate(update_action_idxs):
            action = self.get_action(action_idx_or_action_func=idx)

            #Update signature vars of the retrieved action.
            update_dict = {
                var_type:[params[(var_type, self.var_type)] for params in list_params if (var_type, self.var_type) in params.keys()] for var_type in action.keys()
            }
            action.update_sig_vars(update_dict=update_dict)

            if not has_meaningful_state or count >= update_state_window:
                continue

            #Ensure that there's at least one params in 'list_params' that sampled idx.
            matching_params =  [params for params in list_params if params[(self.var_type,'None')]==idx]
            if not matching_params:
                raise ValueError(f"index: {idx} in 'update_action_idxs' does NOT have any matching params in 'list_params'. Make sure to pass in 'list_params' with at least one params that contain: ({self.var_type},'None')->{idx}")

            #Find the best parameters for this action
            best_params = matching_params[0]

            #Get a fully loadable action
            action = self.get_loadable_action(idx,best_params)

            #Prepare parameters for objective execution
            default_dict = {var_type:val for (var_type,belongs_to),val in best_params.items() if belongs_to == 'None'}
            default_dict[self.var_type] = action

            #Prepare and execute Objective.
            _, model_container = Objective(
                trial=None,
                combined_sampler=None,
                device=device,
                default_dict=default_dict
            ).execute_objective()

            #Update the action's state with the model_container.
            action.update_state(obj=model_container)

#Test: ActionVar

In [ ]:
#Assuming Var, Action, ModelAction, and ActionVar are defined as per the improved code.

#Fake Objective class
class Objective:
    def __init__(self, trial, combined_sampler, device, default_dict):
        self.trial = trial
        self.combined_sampler = combined_sampler
        self.device = device
        self.default_dict = default_dict

    def execute_objective(self):
        model = self.default_dict['model_action'](device=self.device)

        for param in model.parameters():
            param.data.fill_(0)

        model_container = Mock(spec=Learner)
        model_container.model = model

        return (0, model)

#Sample model classes
class ModelDefault(nn.Module):
    def __init__(self, rand_number: int):
        super().__init__()
        self.net = nn.Linear(16, 10)
        self.rand_number = torch.tensor(rand_number)

    def forward(self, x):
        return self.net(x) + self.rand_number

class ModelDropout(nn.Module):
    def __init__(self, activation: nn.Module, dropout_p: float):
        super().__init__()
        self.activation = activation
        self.dropout_p = dropout_p
        self.net = nn.Sequential(
            nn.Linear(16, 10),
            activation(),
            nn.Dropout(p=dropout_p),
            nn.Linear(10, 10)
        )

    def forward(self, x):
        return self.net(x)

#model_action functions
def model_action_default(rand_number: int, device: torch.device):
    return ModelDefault(rand_number=rand_number).to(device)

def model_action_dropout(activation, dropout_p, device: torch.device):
    return ModelDropout(activation=activation, dropout_p=dropout_p).to(device)

#dls_action functions
def dls_action_default(batch_size: int, device: torch.device):
    return {'batch_size': batch_size, 'device': device}

def dls_action_normalize(batch_size: int, normalize: bool):
    return {'batch_size': batch_size, 'normalize': normalize}

def dls_action_activation(batch_size: int, normalize: bool, activation: nn.Module):
    return {'batch_size': batch_size, 'normalize': normalize, 'activation': activation}

class TestActionVar(unittest.TestCase):
    def setUp(self):
        #Define Var objects for parameters of 'model_actions'.
        rand_number_var = Var(
            var_type='rand_number',
            source={'params': {'choices': list(range(1, 11))}, 'sample': 'categorical'},
            belongs_to='model_action'
        )
        activation_var = Var(
            var_type='activation',
            source={'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]}, 'sample': 'categorical'},
            belongs_to='model_action'
        )
        dropout_p_var = Var(
            var_type='dropout_p',
            source={'params': {'low': 0.0, 'high': 1.0}, 'sample': 'float'},
            belongs_to='model_action'
        )
        #Define Var objects for parameters of 'dls_actions'.
        batch_size_var_default = Var(
            var_type='batch_size',
            source={'params': {'low': 16, 'high': 64}, 'sample': 'int'},
            belongs_to='dls_action'
        )
        batch_size_var_normalize = Var(
            var_type='batch_size',
            source={'params': {'low': 16, 'high': 64}, 'sample': 'int'},
            belongs_to='dls_action'
        )
        batch_size_var_activation = Var(
            var_type='batch_size',
            source={'params': {'low': 16, 'high': 64}, 'sample': 'int'},
            belongs_to='dls_action'
        )
        normalize_var_normalize = Var(
            var_type='normalize',
            source={'params': {'choices': [True, False]}, 'sample': 'categorical'},
            belongs_to='dls_action'
        )
        normalize_var_activation = Var(
            var_type='normalize',
            source={'params': {'choices': [True, False]}, 'sample': 'categorical'},
            belongs_to='dls_action'
        )
        activation_var_activation = Var(
            var_type='activation',
            source={'params': {'choices': [nn.ReLU, nn.SiLU]}, 'sample': 'categorical'},
            belongs_to='dls_action'
        )

        #Create ModelAction objects
        self.action_default = ModelAction(
            action_func=model_action_default,
            sig_dict={'rand_number': rand_number_var}
        )
        self.action_dropout = ModelAction(
            action_func=model_action_dropout,
            sig_dict={'activation': activation_var, 'dropout_p': dropout_p_var}
        )
        #Create an ActionVar ModelAction(s).
        self.model_action_var = ActionVar(
            var_type='model_action',
            source={
                'params': {'choices': [self.action_default, self.action_dropout]},
                'sample': 'categorical'
            }
        )

        #Create Action objects for non-stateful actions
        self.dls_action_default = Action(
            action_func=dls_action_default,
            sig_dict={'batch_size': batch_size_var_default}
        )
        self.dls_action_normalize = Action(
            action_func=dls_action_normalize,
            sig_dict={'batch_size': batch_size_var_normalize, 'normalize': normalize_var_normalize}
        )
        self.dls_action_activation = Action(
            action_func=dls_action_activation,
            sig_dict={'batch_size': batch_size_var_activation, 'normalize': normalize_var_activation, 'activation': activation_var_activation}
        )
        #Create an ActionVar for these Actions
        self.dls_action_var = ActionVar(
            var_type='dls_action',
            source={
                'params': {'choices': [self.dls_action_default, self.dls_action_normalize, self.dls_action_activation]},
                'sample': 'categorical'
            }
        )

    def test_model_action_var_init(self):
        #Test initialization of the model_action_var
        self.assertEqual(self.model_action_var.var_type, 'model_action')
        self.assertEqual(self.model_action_var.sample_method, 'categorical')
        self.assertEqual(len(self.model_action_var._frozen_dist), 2)
        self.assertIsInstance(self.model_action_var._frozen_dist[0], ModelAction)
        self.assertIsInstance(self.model_action_var._frozen_dist[1], ModelAction)

    def test_dls_action_var_init(self):
        #Test initialization of the dls_action_var
        self.assertEqual(self.dls_action_var.var_type, 'dls_action')
        self.assertEqual(self.dls_action_var.sample_method, 'categorical')
        self.assertEqual(len(self.dls_action_var._frozen_dist), 3)
        self.assertIsInstance(self.dls_action_var._frozen_dist[0], Action)
        self.assertIsInstance(self.dls_action_var._frozen_dist[1], Action)
        self.assertIsInstance(self.dls_action_var._frozen_dist[2], Action)

    def test_model_is_fixed(self):
        #Initially, the ActionVar is not fixed
        self.assertFalse(self.model_action_var.is_fixed())

        #Fix all variables in the first action
        for var in self.model_action_var._frozen_dist[0].values():
            var._dist = [0]

        #Update the distribution to only include the first action
        self.model_action_var._dist = [0]
        self.assertTrue(self.model_action_var.is_fixed())

    def test_dls_is_fixed(self):
        #Initially, the ActionVar is not fixed
        self.assertFalse(self.dls_action_var.is_fixed())

        #Fix all variables in the first action
        for var in self.dls_action_var._frozen_dist[0].values():
            if var.sample_method == 'categorical':
                var._dist = [0]
            else:
                var._dist['low'] = var._dist['high']

        #Update the distribution to only include the first action
        self.dls_action_var._dist = [0]
        try:
            self.assertTrue(self.dls_action_var.is_fixed())
        except Exception as e:
            raise RuntimeError(f"Distribution: [{self.dls_action_var.distribution()[0].action_func}], ")

    def test_model_get_action(self):
        #Get action by index
        action = self.model_action_var.get_action(0)
        self.assertEqual(action, self.action_default)

        #Get action by function
        action = self.model_action_var.get_action(model_action_dropout)
        self.assertEqual(action, self.action_dropout)

        #Test for KeyError when action not found
        with self.assertRaises(KeyError):
            self.model_action_var.get_action(lambda x: x)
        with self.assertRaises(KeyError):
            self.model_action_var.get_action(2)

    def test_dls_get_action(self):
        #Get action by index
        action = self.dls_action_var.get_action(0)
        self.assertEqual(action, self.dls_action_default)

        #Get action by function
        action = self.dls_action_var.get_action(dls_action_normalize)
        self.assertEqual(action, self.dls_action_normalize)

        #Test for KeyError when action not found
        with self.assertRaises(KeyError):
            self.dls_action_var.get_action(lambda x: x)

    def test_model_get_var(self):
        #Retrieve 'rand_number'
        var = self.model_action_var.get_var(0, 'rand_number')
        self.assertEqual(var.var_type, 'rand_number')
        var = self.model_action_var.get_var(model_action_default, 'rand_number')
        self.assertEqual(var.var_type, 'rand_number')

        #Retrieve 'dropout_p'
        var = self.model_action_var.get_var(1, 'dropout_p')
        self.assertEqual(var.var_type, 'dropout_p')
        var = self.model_action_var.get_var(model_action_dropout, 'dropout_p')
        self.assertEqual(var.var_type, 'dropout_p')

        #Retrieve 'activation'
        var = self.model_action_var.get_var(1, 'activation')
        self.assertEqual(var.var_type, 'activation')
        var = self.model_action_var.get_var(model_action_dropout, 'activation')
        self.assertEqual(var.var_type, 'activation')

    def test_dls_get_var(self):
        #Retrieve 'batch_size' from all actions
        for idx in range(3):
            var = self.dls_action_var.get_var(idx, 'batch_size')
            self.assertEqual(var.var_type, 'batch_size')

        #'normalize' exists in actions at dls_action_normalize, dls_action_dropout.
        var = self.dls_action_var.get_var(1, 'normalize')
        self.assertEqual(var.var_type, 'normalize')

        var = self.dls_action_var.get_var(2, 'normalize')
        self.assertEqual(var.var_type, 'normalize')

        #'activation' exists only in action at dls_action_activation.
        var = self.dls_action_var.get_var(2, 'activation')
        self.assertEqual(var.var_type, 'activation')

        #Test KeyError for specification of wrong sig_var.
        with self.assertRaises(KeyError):
            self.dls_action_var.get_var(0, 'normalize')
        with self.assertRaises(KeyError):
            self.dls_action_var.get_var(1, 'activation')

    def test_get_loadable_action_model_action_var(self):
        #Create a FrozenTrial with fixed parameters
        frozen_trial = optuna.trial.FrozenTrial(
            number=0,  #You can set any trial number.
            trial_id=0,  #You can set any trial id.
            value=None,  #Objective value (set if you have an objective function)
            datetime_start=None,  # Start time of the trial
            datetime_complete=None,  # Completion time of the trial
            params={
                ('rand_number', 'model_action'): 5,  # Index 5 <-> Real value 6.
                ('activation', 'model_action'): 0,   # Index of nn.ReLU
                ('dropout_p', 'model_action'): 0.5,
                ('model_action', 'None'): 1,          # Index of action_dropout
                ('batch_size', 'dls_action'): 1,
                ('normalize', 'dls_action'): 0,
                ('activation', 'dls_action'): 1
            },
            distributions={
                ('rand_number', 'model_action'): optuna.distributions.CategoricalDistribution(choices=list(range(1, 11))),
                ('activation', 'model_action'): optuna.distributions.CategoricalDistribution(choices=[0, 1, 2]),
                ('dropout_p', 'model_action'): optuna.distributions.FloatDistribution(low=0.0, high=1.0),
                ('model_action', 'None'): optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                ('batch_size', 'dls_action'): optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                ('normalize', 'dls_action'): optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                ('activation', 'dls_action'): optuna.distributions.CategoricalDistribution(choices=[0, 1])
            },
            user_attrs={},  # Optional: any user-defined attributes
            system_attrs={},  # Optional: system-specific attributes
            intermediate_values={},  # Optional: store intermediate objective values
            state=optuna.trial.TrialState.COMPLETE,  # State of the trial
            values=None,  # Optional: list of objective values for multi-objective optimization
        )

        #Test action(model_action_default) retrieved from .get_loadable_action
        action = self.model_action_var.get_loadable_action(0, frozen_trial)
        self.assertIsNotNone(action._model_loader)
        self.assertEqual(action._model_loader.keywords['rand_number'], 6)
        action_two = self.model_action_var.get_loadable_action(model_action_default, frozen_trial)
        self.assertEqual(action, action_two)
        action_three = self.model_action_var.get_loadable_action(self.model_action_var.get_action(0), frozen_trial)
        self.assertEqual(action, action_three)
        #Test model created by action(model_action_default)
        model = action(device=None)
        self.assertIsInstance(model, nn.Module)
        self.assertEqual(model.rand_number.item(), 6)
        Y = model(torch.randn(32, 16))
        self.assertEqual(Y.shape, (32, 10))

        #Test action(model_action_dropout) retrieved from .get_loadable_action
        action = self.model_action_var.get_loadable_action(1, frozen_trial)
        self.assertIsNotNone(action._model_loader)
        self.assertEqual(action._model_loader.keywords['activation'], nn.ReLU)
        self.assertEqual(action._model_loader.keywords['dropout_p'], 0.5)
        action_two = self.model_action_var.get_loadable_action(model_action_dropout, frozen_trial)
        self.assertEqual(action, action_two)
        action_three = self.model_action_var.get_loadable_action(self.model_action_var.get_action(1), frozen_trial)
        self.assertEqual(action, action_three)

        #Test model created by action(model_action_dropout)
        model = action(device=None)
        self.assertIsInstance(model, nn.Module)
        self.assertEqual(model.activation, nn.ReLU)
        self.assertEqual(model.dropout_p, 0.5)
        Y = model(torch.randn(32, 16))
        self.assertEqual(Y.shape, (32, 10))

    def test_update_actions_model_action_var(self):
        #Prepare list_params
        list_params = [
            #rand_number: 4
            {('model_action', 'None'): 0, ('rand_number', 'model_action'): 3},
            #activation: nn.SiLU, dropout_p: 0.25.
            {('model_action', 'None'): 1, ('activation', 'model_action'): 2, ('dropout_p', 'model_action'): 0.25},
            #rand_number: 1
            {('model_action', 'None'): 0, ('rand_number', 'model_action'): 0},
            #rand_number: 1
            {('model_action', 'None'): 0, ('rand_number', 'model_action'): 0},
            #activation: nn.PReLU, dropout_p: 0.112.
            {('model_action', 'None'): 1, ('activation', 'model_action'): 1, ('dropout_p', 'model_action'): 0.112},
            #activation: nn.ReLU, dropout_p: 0.998.
            {('model_action', 'None'): 1, ('activation', 'model_action'): 0, ('dropout_p', 'model_action'): 0.998},
        ]

        for i in range(3):
            #Update actions
            self.model_action_var.update_actions([0, 1], list_params, update_state_window=i, device=torch.device('cpu'))

            #Verify updated configuration spaces.
            self.assertEqual(self.model_action_var.distribution(), [self.action_default, self.action_dropout])
            self.assertEqual(self.model_action_var.get_var(0, 'rand_number').distribution(), [1,1,4])
            self.assertEqual(self.model_action_var.get_var(1, 'activation').distribution(), [nn.ReLU,nn.PReLU,nn.SiLU])
            self.assertEqual(self.model_action_var.get_var(1, 'dropout_p').distribution(), {'low':0.112,'high':0.998})

            #action_default <- Action(model_action_default)
            action_default = self.model_action_var.get_action(0)
            #action_dropout <- Action(model_action_dropout)
            action_dropout = self.model_action_var.get_action(1)

            #Test the saved action states
            if i==0:
                self.assertIsNone(action_default.action_state)
                self.assertIsNone(action_dropout.action_state)
            elif i==1:
                self.assertIsNotNone(action_default.action_state)
                self.assertIsNone(action_dropout.action_state)
            else:
                self.assertIsNotNone(action_default.action_state)
                self.assertIsNotNone(action_dropout.action_state)
            action_default.action_state = None
            action_dropout.action_state = None

    def test_error_handling_in_get_loadable_action(self):
        #Test with missing parameters
        params = {('model_action', 'None'): 0}
        with self.assertRaises(KeyError):
            self.model_action_var.get_loadable_action(0, params)

    def test_error_handling_in_update_actions(self):
        #Prepare list_params with missing keys
        list_params = [
            {('model_action', 'None'): 1, ('dropout_p', 'model_action'): 0.112}
        ]
        #Should raise KeyError in .get_action() part of called INSIDE the .update_actions()
        with self.assertRaises(KeyError):
            self.model_action_var.update_actions([1], list_params, update_state_window=10, device=torch.device('cpu'))

        #Prepare list_params that only sampled model_action->action_dropout.
        list_params = [
            {('model_action', 'None'): 1, ('dropout_p', 'model_action'): 0.112, ('activation', 'model_action'): 0},
            {('model_action', 'None'): 1, ('dropout_p', 'model_action'): 1.0, ('activation', 'model_action'): 2},
        ]
        #Pass an unmatching update_indices == [0,1]. This should raise a ValueError.
        with self.assertRaises(ValueError):
            self.model_action_var.update_actions([0,1], list_params, update_state_window=10, device=torch.device('cpu'))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

#Class: SampleCallVisitor, ConfigSpace, UserConfigSpace

In [101]:
import textwrap
#Define a custom AST NodeVisitor to collect var_types
class SampleCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self.user_requests = set()
        self.user_requests_with_defaults = set()

    def visit_Call(self, node):
        if isinstance(node.func, ast.Attribute) and node.func.attr == 'sample' and \
           isinstance(node.func.value, ast.Attribute) and node.func.value.attr == 'self':
            target = node.func.value.value
            msg = target.id if isinstance(target, ast.Name) else target.attr
            raise ValueError(f"Following code sippet '...{target}.self.sample' is not allowed. 'self' keyword cannot be used for attribute access")

        if isinstance(node.func, ast.Attribute) and node.func.attr == 'sample' and \
           isinstance(node.func.value, ast.Name) and node.func.value.id == 'self':
            if not node.args:
                raise ValueError(f"Call self.sample with no argument detected. self.sample must recieve a string argument 'var_type' and an optional literal argument 'default_val'.")
            if len(node.args) > 2:
                raise ValueError(f"Call self.sample with more than 2 arguments detected. self.sample must recieve a string argument 'var_type' and an optional literal argument 'default_val'.")

            var_type_node, default_val = node.args[0], None if len(node.args)==1 else node.args[1]
            if not isinstance(var_type_node, ast.Constant) or not isinstance(var_type_node.value, str):
                raise ValueError(f"Call self.sample with non-string literal argument: '{var_type_node}' detected. self.sample must recieve a string argument 'var_type' and an optional literal argument 'default_val'.")

            #We only check the accuracy of the node IFF 'default_val' is None or literal(ast.Constant)
            #The rest of the cases will be checked during runtime.
            if default_val is None or isinstance(default_val, ast.Constant):
                default_val = default_val.value if default_val else None
                cls = self.user_requests if default_val is None else self.user_requests_with_defaults
                cls.add((var_type_node.value, default_val))

        #Continue walking the tree
        self.generic_visit(node)

In [102]:
class ConfigSpace(ABC):
    """
    ConfigSpace is the user interface for building a properly-formatted configuration space, to be used by the AutoHPO process.
    (1)Allows the user to add,modify,retrieve configuration hyperparameters(whose syntax is identical to that of optuna.sample*)
    (2)Allows the user to retrieve Var/ActionVar object(s).
    (3)Requires the user to subclass it AND order to overwrite 'objective' in the process.
       Additionally, it provides .verify_on_objective() the alignment between configuration space AND the objective function.
    """
    def __init__(self, init_config=None):
        """
        Initialize ConfigSpace.

        Parameters:
        config_space(optional): Another ConfigSpace
        """
        if init_config is not None and not isinstance(init_config, (dict, ConfigSpace)):
            raise ValueError(f"Input 'init_config' must either be None or of type in (dict, ConfigSpace) but found type(init_config): {type(init_config)}")

        self._config = {}
        if isinstance(init_config, ConfigSpace):
            self._config = init_config._config.copy()
            #Inherit the objective fuction as well.
            setattr(self, 'objective', init_config.objective.__func__)
            self.verify_syntax(warn_missing_sig_var_types=True)
            return
        #IF init_config is a dictionary, then shallow-copy it.
        init_config = {} if init_config is None else init_config.copy()

        #Now, we're guaranteed that 'init_config' is a dictionary, and self._config = {} for now.
        #(0)Tools for construction
        init_keys = set(init_config.keys())
        action_keys = {key for key in init_config.keys() if key[0].endswith('--action')}
        non_action_keys = {key for key in init_keys - action_keys if key[1] is None}
        sig_keys = init_keys - action_keys - non_action_keys

        #(1)Verify the syntax/errors in the dictionary 'init_config'
        invalid_action_keys = {key for key in init_keys if '--action' in key[0] and key not in action_keys}
        if invalid_action_keys:
            raise ValueError(f"'--action' suffix can only be appended at the end of each var_type to signify that it's an action parameter but wrongly formatted found: {invalid_action_keys}")
        action_var_types = {var_type[:-len('--action')] for var_type, _ in action_keys}
        non_action_var_types = {var_type for var_type, _ in non_action_keys}
        overlap_var_types = action_var_types & non_action_var_types
        if overlap_var_types:
            overlap_keys = filter(
                lambda key: (key[0] in overlap_var_types) or (key[0][:-len('--action')] in overlap_var_types),
                action_keys | non_action_keys
            )
            raise ValueError(f"Non-action hyperparameters and action hyperparameters cannot share the same var_type but overlap found: {list(overlap_keys)}")
        invalid_sample_methods = {var_type:source for var_type, source in init_config.items() if source['sample'] not in ('categorical','int','float')}
        if invalid_sample_methods:
            raise ValueError(f"All values in 'init_config' must have 'sample' set to one of ('categorical','int','float') but found: {invalid_sample_methods}")

        #(2)Transfer the info from the dictionary to ConfigSpace.
        #Firstly, define all the action parameters
        for key in action_keys:
            source = init_config[key]
            sample_method = source['sample']
            params = source['params']
            if sample_method!='categorical':
                raise ValueError(f"init_config[{key}]['sample'] must be set to 'categorical' but found: {sample_method}")
            action_var_type, belongs_to = key[0][:-len('--action')] , key[1]
            self.set_categorical(
                var_type=action_var_type,
                belongs_to=belongs_to,
                action=True,
                **params
            )
        ordered_keys = [key for key in non_action_keys] + [key for key in sig_keys]
        #Secondly, define all the non-action parameters and sig parameters, IN ORDER.
        for key in ordered_keys:
            source = init_config[key]
            sample_method = source['sample']
            params = source['params']
            var_type, belongs_to = key
            cls = self.set_int if sample_method == 'int' \
                               else self.set_float if sample_method == 'float' \
                               else partial(self.set_categorical, action=False)
            cls(var_type=var_type, belongs_to=belongs_to, **params)

        #(3)Perform post verification
        self.verify_syntax(True)

    def __len__(self):
        return len(self._config)

    def verify_objective(self, warn_missing_var_types: bool):
        """
        -Parameter: warn_missing_var_types := Whether to raise Warning instead of Errors for missing variables.
                    Can be set to True if user is actively adjusting the ConfigSpace.
        """
        #Get the source code of the 'self.objective'
        try:
            source = inspect.getsource(self.objective)
        except OSError as e:
            raise RuntimeError("Unable to retrieve source code of '.objective'"
                                "Ensure that the method is defined in a way that allows source code inspection.") from e

        #Dedent the source code
        source = textwrap.dedent(source)
        #Parse the source code into an AST
        tree = ast.parse(source)
        #Create an instance of the visitor and walk the tree
        visitor = SampleCallVisitor()
        visitor.visit(tree)

        config_var_types, sig_var_types = self.var_types(), self.sig_var_types()
        #Extract only the user requests that have either None or an instance of ast.Constant
        user_requests = visitor.user_requests
        user_requests_with_defaults = visitor.user_requests_with_defaults
        total_requests = user_requests | user_requests_with_defaults

        #Catch any illegal request(s) to signature parameters.
        #NOTE: Overlap between config_var_types and sig_var_types IS allowed,
        #      and we only check incidents for sig_var_types - config_var_types
        illegal_sig_var_types = sig_var_types - config_var_types
        illegal_sig_calls = set(filter(
            lambda req: req[0] in illegal_sig_var_types,
            total_requests
        ))
        if illegal_sig_calls:
            formatted = [f'self.sample({var_type},{default_val})' for var_type,default_val in illegal_sig_calls]
            raise SyntaxError(f"Following calls in '.objective' are illegal due to var_type referring to a signature parameter: {formatted}")

        #If independent user_requests are not covered by config_space.var_types(),
        #then EITHER raise ValueError or raise Warning.
        calls_with_missing_var_types = {req for req in user_requests if req[0] not in config_var_types}
        if calls_with_missing_var_types:
            formatted = [f'self.sample({req[0]},{req[1]})' for req in calls_with_missing_var_types]
            msg = f"Following calls in '.objective' are illegal due to var_type referring to a parameter that does not exist in the configuration space: {formatted}"
            if not warn_missing_var_types:
                raise ValueError(msg)
            warnings.warn(msg)

        #If non-independent user_requests are made with var_types not covered by config_space.var_types(),
        #then throw warning.
        calls_with_potentially_missing_var_types = {
            req for req in user_requests_with_defaults
            if req[0] not in config_var_types
        }
        if calls_with_potentially_missing_var_types:
            formatted = [f'self.sample({req[0]},{req[1]})' for req in calls_with_potentially_missing_var_types]
            warnings.warn(f"Following calls in '.objective' are referring to non-existing var_type(s) and will simply return respective default_val(s): {formatted}.")

        #Catch any (var_type, default_val) such that var_type exists in config_var_types BUT default_val is a value NOT allowed in the configuration space of 'var_type'.
        existent_requests_with_defaults = {req for req in user_requests_with_defaults if req[0] in config_var_types}
        req_to_sources = {req: self.get_source(req[0]) for req in existent_requests_with_defaults}
        requests_with_illegal_defaults = set()
        for (var_type,default_val),source in req_to_sources.items():
            params, sample_method = source['params'], source['sample']
            if any([
                sample_method=='float' and not(isinstance(default_val, (float,int)) and  params['low']<=default_val<=params['high']),
                sample_method=='int' and not(isinstance(default_val, int) and params['low']<=default_val<=params['high']),
                sample_method=='categorical' and default_val not in params['choices']
            ]):
                requests_with_illegal_defaults.add((var_type,default_val))
        if requests_with_illegal_defaults:
            formatted = [f'self.sample({req[0]},{req[1]})' for req in requests_with_illegal_defaults]
            raise ValueError(f"Following calls in '.objective' are illegal due to 'default_val' referring to values that are not allowed in the configuration space of corresponding var_type: {formatted}")

    def verify_syntax(self, warn_missing_sig_var_types: bool):
        """
        Verify the hyperparameter configuration.

        Raises ValueError or SyntaxError depending on the point of failure.
        """
        #First and foremost, ensure that there's no overlap between non-action_var_types AND action_var_types
        overlaps = self.non_action_var_types() & self.action_var_types()
        if overlaps:
            raise ValueError(f"Non-action hyperparameters and action hyperparameters overlap: {overlaps}")
        #Second, check for missing variable types:
        self._verify_missing_sig_var_types(warn_missing_sig_var_types)
        #Add more if needed...

    def _verify_missing_sig_var_types(self, warn_missing_sig_var_types: bool):
        msg = ''
        for key, source in self._config.items():
            var_type, belongs_to = key
            if '--action' in var_type and belongs_to is None:
                action_var_type = var_type[:-len('--action')]
                sig_var_types = set()
                for func in source['params']['choices']:
                    sig_var_types |= set(signature(func).parameters.keys()) - {'device'}

                #Below segment checks for missing parameters. We don't output an error immediately.
                existing_var_types = self.sig_var_types(action_var_type)
                missing_var_types = sig_var_types - existing_var_types
                if missing_var_types:
                    msg += f"Action hyperparameter '{action_var_type}' requires additional hyperparameters: {[(p, action_var_type) for p in missing_var_types]}\n"

        if len(msg) and not warn_missing_sig_var_types: #raise ValueError
            raise ValueError(msg)
        if len(msg): #warn the user.
            warnings.warn(msg)

    def var_types(self) -> set:
        return {var_type[:-len('--action')] if '--action' in var_type else var_type for (var_type, belongs_to) in self._config.keys()
                if belongs_to is None}

    def action_var_types(self) -> set:
        return {var_type[:-len('--action')] for (var_type, _) in self._config.keys() if '--action' in var_type}

    def non_action_var_types(self) -> set:
        return self.var_types() - self.action_var_types()

    def sig_var_types(self, action_var_type: str = None) -> set:
        if action_var_type is None:
            return {var_type for (var_type, belongs_to) in self._config.keys() if belongs_to is not None}
        else:
            action_var_types = self.action_var_types()
            if action_var_type not in action_var_types:
                raise KeyError(f"Input 'action_var_type': {action_var_type} is not in the set of registered action hyperparameters: {action_var_types}")
            return {var_type for (var_type, belongs_to) in self._config.keys() if belongs_to == action_var_type}

    def get_source(self, var_type: str, belongs_to: str = None) -> dict:
        config_keys = self._config.keys()
        var_types = self.var_types()
        action_var_types = self.action_var_types()
        sig_var_types = self.sig_var_types()
        if belongs_to is not None and belongs_to not in action_var_types:
            raise KeyError(f"Input 'belongs_to': {belongs_to} is not in the set of registered action hyperparameters: {action_var_types}")
        if var_type not in var_types and belongs_to is None:
            raise KeyError(f"Input 'var_type': {var_type} is not in the set of registered hyperparameters: {var_types}")
        if var_type in var_types and belongs_to is not None:
            raise KeyError(f"Input 'belongs_to': {belongs_to} should be None if 'var_type': {var_type} is not a signature hyperparameter.")
        if var_type in sig_var_types and (var_type, belongs_to) not in config_keys:
            raise KeyError(f"var_type: {var_type} is not one of the signature hyperparameters of action hyperparameter: {belongs_to}")
        formatted_var_type = var_type+'--action' if var_type in action_var_types else var_type
        if (formatted_var_type, belongs_to) not in config_keys:
            raise KeyError(f"Requested ({var_type}, {belongs_to}) does not exist in the set of all pairs stored in ConfigSpace: {[(p[:-len('--action')] if '--action' in p else p,bt) for p,bt in config_keys]}")

        return self._config[(formatted_var_type, belongs_to)]

    def get_sources(self, list_of_keys) -> list:
        """
        Get the configurations for a list of hyperparameters.

        Parameters:
        - list_of_keys (list or set): List of (var_type, belongs_to) tuples.
        """
        if not isinstance(list_of_keys, (list, set, tuple)):
            raise ValueError(f"Input 'list_of_keys' must be a list,set or a tuple but found: {type(list_of_keys)}")
        if any(not isinstance(key, tuple) for key in list_of_keys):
            raise KeyError(f"Non-tuple elements found in input 'list_of_keys': {list_of_keys}")
        return [self.get_source(var_type=key[0], belongs_to=key[1]) for key in list_of_keys]

    def get_var(self, var_type: str, belongs_to: str = None) -> Var:
        """
        Get the configuration for a specific hyperparameter.

        Parameters:
        - var_type (str): The hyperparameter name.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.
        """
        source = self.get_source(var_type, belongs_to)
        if var_type not in self.action_var_types():
            return Var(var_type, source, belongs_to)

        #Assemble 'sig_dict' to initialize Action/Model objects
        action_var_type = var_type
        choices = source['params']['choices']
        action_choices = []
        for action_func in choices:
            sig = set(signature(action_func).parameters.keys()) - {'device'}
            existing_var_types = self.sig_var_types(action_var_type)
            missing_var_types = sig - existing_var_types
            if missing_var_types:
                raise ValueError(f"Requested action hyperparameter: {action_var_type} cannot be assembled due to missing hyperparameters: {missing_var_types}")
            sig_dict = {
                sig_var_type: Var(sig_var_type, self.get_source(sig_var_type, action_var_type), action_var_type)
                for sig_var_type in sig
            }
            #Add to 'action_choices' the initialized Action.
            action_choices.append(ModelAction(action_func, sig_dict) if 'model' in action_var_type else Action(action_func, sig_dict))
        source = {
            'params': {'choices': action_choices},
            'sample': 'categorical'
        }
        return ActionVar(action_var_type, source)

    def get_action_vars(self, action_var_types=None):
        self.verify_syntax(False)
        if isinstance(action_var_types, str):
            action_var_types = [action_var_types]
        if action_var_types is not None:
            return [self.get_var(action_var_type) for action_var_type in action_var_types]
        return [self.get_var(action_var_type) for action_var_type in self.action_var_types()]

    def get_non_action_vars(self, non_action_var_types=None):
        self.verify_syntax(False)
        if isinstance(non_action_var_types, str):
            non_action_var_types = [non_action_var_types]
        if non_action_var_types is not None:
            return [self.get_var(var_type) for var_type in non_action_var_types]
        return [self.get_var(var_type) for var_type in self.non_action_var_types()]

    def get_sig_vars(self, list_keys: list=None):
        if list_keys is not None and not isinstance(list_keys, list):
            raise ValueError(f"Input 'list_keys' must be None or a list but found: {list_keys}")
        invaid_keys = set() if list_keys is None else {key not in self._config.keys() for key in list_keys}
        if invaid_keys:
            raise KeyError(f"Non-existent keys found in input 'list_keys': {list_keys}")

        if list_keys is None:
            return [self.get_var(*key) for key in self._config.keys() if key[1] is not None]
        return [self.get_var(*key) for key in list_keys]

    def set_float(self, var_type: str, low: float, high: float, step: float = None, log: bool = False, belongs_to: str = None):
        """
        Define a float hyperparameter.

        Parameters:
        - var_type (str): The name of the hyperparameter.
        - low (float): The lower bound.
        - high (float): The upper bound.
        - step (float, optional): Step size.
        - log (bool, optional): Whether to sample logarithmically.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.
        """
        #ValueError(s).
        if any([
            not isinstance(var_type, str),
            not (belongs_to is None or isinstance(belongs_to, str)),
            not isinstance(low, (float, int)),
            not isinstance(high, (float, int)),
            not (step is None or isinstance(step, (float, int))),
            not isinstance(log, bool)
        ]):
            types = [type(x) for x in [var_type, str(belongs_to), low, high, step, log]]
            raise ValueError(f"Input var_type, belongs_to, low, high, step, log should be of types str, str/None, float/int, float/int, float/int or None, bool respectively but found {types}")
        if low > high:
            raise ValueError(f"low: {low} > high: {high}")

        #KeyError(s).
        if any('-' in x or '>' in x for x in [var_type, str(belongs_to)]):
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) contains at least one of the following characters ['-', '>']")
        if belongs_to is not None and belongs_to not in self.action_var_types():
            raise KeyError(f"'belongs_to': {belongs_to} is not in the set of registered action hyperparameters: {self.action_var_types()}")
        if var_type in self.action_var_types():
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) defines a non-action parameter but var_type: {var_type} collides with the var_type of a registered action parameter. Overlapping names between non-action parameters and action parameters is not allowed.")
        belongs_to_choices = self.get_source(belongs_to)['params']['choices'] if belongs_to is not None else []
        partitioned = [set(signature(action_func).parameters.keys()) for action_func in belongs_to_choices]
        potential_sig_var_types = set.union(*partitioned) if partitioned else set()
        if belongs_to is not None and var_type not in potential_sig_var_types:
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) cannot define a signature parameter since var_type: {var_type} does not belong to the {belongs_to}'s signature: {potential_sig_var_types}")

        params = {'low': low, 'high': high, 'log': log}
        if step is not None:
            params['step'] = step

        self._config[(var_type, belongs_to)] = {
            'params': params,
            'sample': 'float'
        }

        self.verify_syntax(True)

    def set_int(self, var_type: str, low: int, high: int, step: int = 1, log: bool = False, belongs_to: str = None):
        """
        Define an integer hyperparameter.

        Parameters:
        - var_type (str): The name of the hyperparameter.
        - low (int): The lower bound.
        - high (int): The upper bound.
        - step (int, optional): Step size.
        - log (bool, optional): Whether to sample logarithmically.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.
        """
        #ValueError(s).
        if any([
            not isinstance(var_type, str),
            not (belongs_to is None or isinstance(belongs_to, str)),
            not isinstance(low, int),
            not isinstance(high, int),
            not (step is None or isinstance(step, int)),
            not isinstance(log, bool)
        ]):
            types = [type(x) for x in [var_type, str(belongs_to), low, high, step, log]]
            raise ValueError(f"Input var_type, belongs_to, low, high, step, log should be of types str, str/None, float/int, float/int, float/int or None, bool respectively but found {types}")
        if low > high:
            raise ValueError(f"low: {low} > high: {high}")

        #KeyError(s).
        if any('-' in x or '>' in x for x in [var_type, str(belongs_to)]):
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) contains at least one of the following characters ['-', '>']")
        if belongs_to is not None and belongs_to not in self.action_var_types():
            raise KeyError(f"'belongs_to': {belongs_to} is not in the set of registered action hyperparameters: {self.action_var_types()}")
        if var_type in self.action_var_types():
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) defines a non-action parameter but var_type: {var_type} collides with the var_type of a registered action parameter. Overlapping names between non-action parameter and action parameters is not allowed.")
        belongs_to_choices = self.get_source(belongs_to)['params']['choices'] if belongs_to is not None else []
        partitioned = [set(signature(action_func).parameters.keys()) for action_func in belongs_to_choices]
        potential_sig_var_types = set.union(*partitioned) if partitioned else set()
        if belongs_to is not None and var_type not in potential_sig_var_types:
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) cannot define a signature parameter since var_type: {var_type} does not belong to the {belongs_to}'s signature: {potential_sig_var_types}")

        params = {'low': low, 'high': high, 'log': log, 'step': step}

        self._config[(var_type, belongs_to)] = {
            'params': params,
            'sample': 'int'
        }

        self.verify_syntax(True)

    def set_categorical(self, var_type: str, choices: list, action: bool = False, belongs_to: str = None):
        """
        Define a categorical hyperparameter.

        Parameters:
        - var_type (str): The name of the hyperparameter.
        - choices (list): List of possible values.
        - action (bool, optional): Whether this hyperparameter is an action hyperparameter.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.
        """
        #ValueError(s).
        if any([
            not isinstance(var_type, str),
            not (belongs_to is None or isinstance(belongs_to, str)),
            not isinstance(action, bool)
        ]):
            types = [type(x) for x in [var_type, str(belongs_to)]]
            raise ValueError(f"Input var_type, belongs_to should be of types str, str/None respectively but found {types}")
        if not isinstance(choices, list):
            raise ValueError(f"Input 'choices' must be a list but found: {type(choices)}")
        if action and any(not callable(choice) for choice in choices):
            raise ValueError(f"Non-callable elements found in input choices: {choices}")

        #KeyError(s).
        if any('-' in x or '>' in x for x in [var_type, str(belongs_to)]):
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) contains at least one of the following characters ['-', '>']")
        if belongs_to is not None and belongs_to not in self.action_var_types():
            raise KeyError(f"'belongs_to': {belongs_to} is not in the set of registered action hyperparameters: {self.action_var_types()}")
        if not action and var_type in self.action_var_types() or action and var_type in self.non_action_var_types():
            target, opposite = ('action','non-action') if action else ('non-action', 'action')
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) defines a {target} parameter but var_type: {var_type} collides with the var_type of a registered {opposite} parameter. Overlapping names between {target} parameters and {opposite} parameters is not allowed.")
        if action and belongs_to is not None:
            raise KeyError(f"Input 'belongs_to' must be None if 'action' is True but found: {belongs_to}")
        belongs_to_choices = self.get_source(belongs_to)['params']['choices'] if belongs_to is not None else []
        partitioned = [set(signature(action_func).parameters.keys()) for action_func in belongs_to_choices]
        potential_sig_var_types = set.union(*partitioned) if partitioned else set()
        if belongs_to is not None and var_type not in potential_sig_var_types:
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) cannot define a signature parameter since var_type: {var_type} does not belong to the {belongs_to}'s signature: {potential_sig_var_types}")

        if not action or (var_type+'--action', belongs_to) not in self._config.keys():
            formatted_var_type = var_type+'--action' if action else var_type
            self._config[(formatted_var_type, belongs_to)] = {
                'params': {'choices': choices},
                'sample': 'categorical'
            }
        else: #Overriding the existing (action_var_type,belongs_to) has been requested:
            #STEP1: POP all the related parameters
            popped = self.pop(var_type)
            #STEP2: Now, given that we're guaranteed we DO NOT have (var_type+'--action',None) in self._config,
            #       we .set_categorical(var_type,..rest)
            self.set_categorical(var_type=var_type, choices=choices, action=True)
            #STEP3: Last but not least, selectively 'inherit' the information by looking at the potential sig_var_types needed for the new action parameter
            partitioned = [set(signature(action_func).parameters.keys()) for action_func in choices]
            sig_var_types = set.union(*partitioned) if partitioned else set()
            inherit_sig_var_keys = {
                sig_var_type for sig_var_type in sig_var_types
                if (sig_var_type,var_type) in popped.keys()
            }
            for key in inherit_sig_var_keys:
                source = popped[key]
                sig_var_type, action_var_type = key
                if source['sample']=='categorical':
                    self.set_categorical(
                        var_type=sig_var_type,
                        belongs_to=action_var_type,
                        choices=source['params']['choices']
                    )
                else:
                    cls = self.set_float if source['sample']=='float' else self.set_int
                    cls(var_type=sig_var_type, belongs_to=action_var_type, **source['params'])

        self.verify_syntax(True)

    def pop(self, var_type: str, belongs_to: str=None):
        """
        Delete a hyperparameter and return its source(s). Multiple sources may be returned if a deleted hyperparameter is an action parameter.

        Parameters:
        - var_type (str): The name of the hyperparameter.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.

        Returns:
        Dictionary(or dictionaries if what you deleted is an action_var_type) of mapping (var_type->source) for all deleted var_types.

        NOTE: If you delete an Action parameter, Then ALL of its signature parameters will also be deleted.
        """
        #If you can successfully retrieve a source from the provided pair, it means we have a valid pair
        source = self.get_source(var_type, belongs_to)
        is_action_var_type = var_type in self.action_var_types()

        popped = dict()
        #(1)If action_var_type with none of the signature parameters, Then just delete it.
        #(2)If action_var_type with signature parameters, Then pop all the signature parameters first, then combine it with the popped action parameter(now action parameter with no signature parameters)
        #(3)If not action_var_type, just pop it out.
        if is_action_var_type and self.sig_var_types(var_type):
            sig_var_types = self.sig_var_types(var_type)
            #Note that we avoid recursion by directly popping both sig_var_types, and the action_var_type.
            popped |= {(sig_var_type, var_type):self._config.pop((sig_var_type, var_type)) for sig_var_type in self.sig_var_types(var_type)}
            return {(var_type,belongs_to):self._config.pop((var_type+'--action',belongs_to))} | popped

        formatted_var_type = var_type+'--action' if is_action_var_type else var_type
        popped = self._config.pop((formatted_var_type, belongs_to))
        #If what was popped was a signature var_type, then run self.verify_syntax() to print out message about what the owner action_var_type will now be missing.
        if belongs_to is not None:
            self.verify_syntax(True)
        #Return the resultant value.
        return {(var_type, belongs_to): popped}

    @abstractmethod
    def objective(self):
        return NotImplementedError(".objective not implemented")

In [103]:
class UserConfigSpace(ConfigSpace):
    def objective(self):
        #dls_action <- Action object encapsulating DataLoaders construction(and involved parameters)
        dls_action = self.sample('dls_action')
        #dls <- Executed dls_action(device=self.device)
        dls = dls_action(device=self.device)
        #model_action <- ModelAction object encapsulating model construction(and involved parameters).
        model_action = self.sample('model_action')
        #model_action_func <- model_action's action_func
        model_action_func = model_action.action_func
        #model_action's saves state <- model_action.action_state.
        model_action_state = model_action.action_state
        #model <- Model yielded from action.
        model = model_action(device=self.device)
        #loss_func <- Sampled loss function.
        loss_func = self.sample('loss_func')
        #If the loss_func is a subclass of nn.Module, Then initialize.
        if isinstance(loss_func, type) and issubclass(loss_func, nn.Module):
            loss_func = loss_func()

        #model_container <- Fast.AI's model_container object constructed from it.
        model_container = Learner(dls=dls,
                        model=model,
                        loss_func=loss_func() if isinstance(loss_func, nn.Module) else loss_func,
                        #***opt_func=self.sample('opt_func'), NOT IMPLEMENTED YET!!!***
                        metrics=self.sample('metric')
                        )

        #Interaction with ModelAction.action_state during runtime to make a decision on 'freeze'.
        if model_action_state is None and self.sample('freeze', False):
            model_container.freeze()
        else: #IF (action_state is not None) OR (action_state is None but sampled 'freeze' states False, THEN .unfreeze().
            model_container.unfreeze()

        #Barebone cbs list:
        cbs = []
        #Fill the list:
        if self.sample('gradient_clip',False): #Apply Gradient clipping, if exists and is True.
            cbs.append(GradientClip(max_norm=self.sample('max_norm', 0.1))) #If exists, then 'max_norm' MUST exist, hence no safety guarantee.
        if self.trial is None:
            cbs.append(EarlyStoppingCallback(monitor='valid_loss',
                                            min_delta=self.sample('min_delta', 0.1),
                                            patience=self.sample('patience', 15)))
        else: #IF trial is a type of optuna.Trail, then apply PruningCallback.
            cbs.append(FastAIPruningCallback(trial=self.trial, monitor='valid_loss'))

        #Make the action and return it.
        trainer = None
        if self.sample('one_cycle', True):
            lr_low = self.sample('lr_low', 9e-4)
            lr_high = self.sample('lr_high', lr_low)
            trainer = partial(model_container.fit_one_cycle,
                            n_epoch=self.sample('n_epoch', 10),
                            lr_max=slice(lr_low, lr_high),
                            wd=self.sample('wd', 0.01), pct_start=self.sample('pct_start',0.3),
                            cbs=cbs)
        else:
            trainer = partial(model_container.fit,
                            n_epoch=self.sample('n_epoch', 10),
                            lr=self.sample('lr', 1e-3),
                            wd=self.sample('wd', 0.01),
                            cbs=cbs)

        #Train the model.
        with model_container.no_bar(), model_container.no_logging():
                trainer()

        #Return (objective loss, model container)
        return (model_container.validate()[0], model_container)

#Test: ConfigSpace, UserConfigSpace

In [ ]:
def dummy_action1(param1, param2):
    pass

def dummy_action2(param3, param4, device=None):
    pass

def dummy_action3(param5, param6):
    pass

def dummy_action_evens(param2, param4, param6, device=None):
    pass

def dummy_action_odds(param1, param3, param5, device=None):
    pass

def dummy_action_combined(param1, param2, param3, param4, param5, param6):
    pass

def non_callable():
    pass

class TestUserConfigSpace(unittest.TestCase):
    def setUp(self):
        self.cs = UserConfigSpace()

    def test_set_float(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        expected = {
            ('learning_rate', None): {
                'params': {'low': 0.001, 'high': 0.01, 'log': False},
                'sample': 'float'
            }
        }
        self.assertEqual(self.cs._config, expected)

        self.assertEqual(
            self.cs.get_source('learning_rate'),
            expected[('learning_rate', None)]
        )

    def test_set_int(self):
        self.cs.set_int('num_layers', low=1, high=5)
        expected = {
            ('num_layers', None): {
                'params': {'low': 1, 'high': 5, 'log': False, 'step': 1},
                'sample': 'int'
            }
        }
        self.assertEqual(self.cs._config, expected)

        self.assertEqual(
            self.cs.get_source('num_layers'),
            expected[('num_layers', None)]
        )

    def test_set_categorical(self):
        self.cs.set_categorical('activation', choices=[nn.ReLU, nn.Tanh, nn.Sigmoid])
        expected = {
            ('activation', None): {
                'params': {'choices': [nn.ReLU, nn.Tanh, nn.Sigmoid]},
                'sample': 'categorical'
            }
        }
        self.assertEqual(self.cs._config, expected)

        self.assertEqual(
            self.cs.get_source('activation'),
            expected[('activation', None)]
        )

    def test_set_action(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1, dummy_action2], action=True)
        expected = {
            ('optimizer--action', None): {
                'params': {'choices': [dummy_action1, dummy_action2]},
                'sample': 'categorical'
            }
        }
        self.assertEqual(self.cs._config, expected)

        self.assertEqual(
            self.cs.get_source('optimizer'),
            expected[('optimizer--action', None)]
        )

        for i in range(1,5):
            self.cs.set_float(f'param{i}', low=0.0, high=1.0, belongs_to='optimizer')
        sig_dict_dummy_action1 = {f'param{i}': Var(f'param{i}', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer') for i in range(1,3)}
        sig_dict_dummy_action2 = {f'param{i}': Var(f'param{i}', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer') for i in range(3,5)}
        self.assertEqual(
            self.cs.get_var('optimizer'),
            ActionVar(
                'optimizer',
                 { 'params': {'choices': [Action(dummy_action1, sig_dict_dummy_action1), Action(dummy_action2, sig_dict_dummy_action2)]},
                   'sample': 'categorical'}
            )
        )

    def test_categorization(self):
        #action_var_types
        self.cs.set_categorical('optimizer', choices=[dummy_action1, dummy_action2], action=True)

        #non_action_var_types
        self.cs.set_int('num_layers', low=1, high=5)
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        self.cs.set_categorical('activation', choices=[nn.ReLU, nn.Tanh, nn.SiLU])

        #sig_var_types
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param3', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer')

        self.assertEqual(
            self.cs.action_var_types(),
            {'optimizer'}
        )
        self.assertEqual(
            self.cs.non_action_var_types(),
            {'num_layers', 'learning_rate', 'activation'}
        )
        self.assertEqual(
            self.cs.sig_var_types('optimizer'),
            {'param1', 'param2', 'param3', 'param4'}
        )

        with self.assertRaises(KeyError):
            self.cs.set_float('param5', low=0.0, high=1.0, belongs_to='non_existent_action')
        with self.assertRaises(KeyError):
            #Should raise KeyError since 'non_existent_param' does NOT exist in signature(s) of ActionVar category 'optimizer'
            self.cs.set_int('non_existent_param', low=0, high=1, belongs_to='optimizer')

        #Get rid of the 'param1' from 'optimizer'
        popped = self.cs.pop('param1', 'optimizer')
        self.assertEqual(
            popped,
            {
                ('param1', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                }
            }
        )

        #Verify that the action parameter requires signature parameters now.(The missing one; param1)
        with self.assertRaises(ValueError):
            self.cs.verify_syntax(False)

    def test_add_signature_parameters(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1, dummy_action2], action=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param3', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer')
        #Now verify should pass
        self.cs.verify_syntax(False)

        #Delete the 'optimizer'. This should delete all the other 'param1',...,'param4' as well.
        popped = self.cs.pop('optimizer')
        expected = {
                ('optimizer', None): {
                    'params': {'choices': [dummy_action1, dummy_action2]},
                    'sample': 'categorical'
                },
                ('param1', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                },
                ('param2', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                },
                ('param3', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                },
                ('param4', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                }
        }
        self.assertEqual(
            popped,
            expected,
            msg = f"popped: {popped} != \nexpected: {expected}"
        )

    def test_invalid_var_type_name(self):
        with self.assertRaises(KeyError):
            self.cs.set_float('learning>rate', low=0.001, high=0.01)

    def test_invalid_belongs_to(self):
        with self.assertRaises(KeyError):
            self.cs.set_float('param', low=0.0, high=1.0, belongs_to='non_existent_action')

    def test_non_callable_action_choice(self):
        with self.assertRaises(ValueError): #action=True means that all inputs in 'choices' must be callable.
            self.cs.set_categorical('optimizer', choices=[1,2], action=True)

    def test_set_categorical_with_invalid_choices(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('optimizer', choices='not_a_list')

    def test_set_float_with_invalid_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_float('learning_rate', low='0.001', high=0.01)

    def test_set_int_with_invalid_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_int('num_layers', low=1, high='5')

    def test_get_non_action_var(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        var = self.cs.get_var('learning_rate')
        self.assertIsInstance(var, Var)
        expected_var = Var('learning_rate', {'params': {'low': 0.001, 'high': 0.01, 'log': False}, 'sample': 'float'}, None)
        self.assertEqual(var, expected_var)

    def test_get_action_var(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        action_var = self.cs.get_var('optimizer')
        self.assertIsInstance(action_var, ActionVar)
        expected_param1_var = Var('param1', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_param2_var = Var('param2', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_action_var = ActionVar(
            'optimizer',
             {
                 'params': {'choices': [Action(dummy_action1, {expected_param1_var.var_type: expected_param1_var, expected_param2_var.var_type: expected_param2_var})]},
                 'sample': 'categorical'
             }
        )
        self.assertEqual(
            action_var,
            expected_action_var,
            msg=f"action_var: {action_var} != \nexpected_action_var: {expected_action_var}"
        )

    def test_get_action_vars(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        #Attempt to retrieve optimizer w/o filling in all the required signature parameters
        with self.assertRaises(ValueError): #Will internally run self._verify(), NOT self._verify(True,True)
            self.cs.get_action_vars()

        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.verify_syntax(False)

        #This time, you'll be missing param1
        self.cs.pop('param1', 'optimizer')
        with self.assertRaises(ValueError):
            self.cs.get_action_vars()

        #Add it again.
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        action_vars = self.cs.get_action_vars() #Should work this time.

        #Verify the number of action_vars.
        self.assertEqual(len(action_vars), 1)

        #Verify that it's as we expected.
        action_var = action_vars[0]
        self.assertIsInstance(action_var, ActionVar)
        expected_param1_var = Var('param1', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_param2_var = Var('param2', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_action_var = ActionVar(
            'optimizer',
             {
                 'params': {'choices': [Action(dummy_action1, {expected_param1_var.var_type: expected_param1_var, expected_param2_var.var_type: expected_param2_var})]},
                 'sample': 'categorical'
             }
        )
        self.assertEqual(
            action_var,
            expected_action_var,
            msg=f"action_var: {action_var} != \nexpected_action_var: {expected_action_var}"
        )

    def test_get_non_action_vars(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        self.cs.set_int('num_layers', low=1, high=5)
        vars = self.cs.get_non_action_vars()
        self.assertEqual(len(vars), 2)

        #Check that created Var objects are as we expected.
        expected_lr_var = Var('learning_rate', {'params': {'low': 0.001, 'high': 0.01, 'log': False}, 'sample': 'float'}, None)
        expected_num_layers_var = Var('num_layers', {'params': {'low': 1, 'high': 5, 'log': False, 'step': 1}, 'sample': 'int'}, None)
        expected_dict = {expected_lr_var.var_type: expected_lr_var, expected_num_layers_var.var_type: expected_num_layers_var}
        for var in vars:
            var_type = var.var_type
            expected = expected_dict[var_type]
            self.assertEqual(
                var,
                expected,
                msg=f"var_type: {var_type} maps to --> var: {var} != /nexpected: {expected}"
            )
        #Do the same, but for signature parameters. Don't forget to add the 'optimizer' first.
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')

        with self.assertRaises(KeyError): #'param4' not in optimizer signatures
            self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer')

        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        sig_vars = self.cs.get_sig_vars()
        self.assertEqual(len(sig_vars), 2)
        expected_param1_var = Var('param1', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_param2_var = Var('param2', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_dict = {expected_param1_var.var_type:expected_param1_var,
                         expected_param2_var.var_type:expected_param2_var}
        for sig_var in sig_vars:
            sig_var_type = sig_var.var_type
            expected = expected_dict[sig_var_type]
            self.assertEqual(
                sig_var,
                expected,
                msg=f"sig_var_type: {sig_var_type} maps to --> sig_var: {sig_var} != /nexpected: {expected}"
            )

    def test_post_set_verify(self):
        #Just print what you need out.
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)

        #Should NOT raise KeyError, overlap between non-action_var_types and sig_var_types is ALLOWED.
        self.cs.set_float('param1', low=0.0, high=1.0)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.verify_syntax(False)

        #Check sig_var_types
        sig_var_types = self.cs.sig_var_types()
        self.assertEqual(
            sig_var_types,
            {'param1', 'param2'}
        )

        #Check for action_var_types
        action_var_types = self.cs.action_var_types()
        self.assertEqual(
            action_var_types,
            {'optimizer'}
        )

        #Check for non_action_var_types
        non_action_var_types = self.cs.non_action_var_types()
        self.assertEqual(
            non_action_var_types,
            {'param1'}
        )

    def test_duplicate_hyperparameter(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        self.cs.set_float('learning_rate', low=0.0001, high=0.1) #overriding is ok!
        self.assertTrue(True)

    def test_sig_var_types_method(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_categorical('optimizer2', choices=[dummy_action2], action=True)

        with self.assertRaises(KeyError):
            self.cs.set_float('param3', low=0.0, high=1.0, belongs_to='optimizer') #ONLY param1,param2 allowed
        with self.assertRaises(KeyError):
            self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer') #ONLY param1,param2 allowed

        self.cs.set_float('param3', low=0.0, high=1.0, belongs_to='optimizer2')
        self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer2')

        sig_var_types = self.cs.sig_var_types('optimizer')
        self.assertEqual(sig_var_types, {'param1', 'param2'})

        sig_var_types = self.cs.sig_var_types('optimizer2')
        self.assertEqual(sig_var_types, {'param3', 'param4'})

        sig_var_types = self.cs.sig_var_types()
        self.assertEqual(sig_var_types, {'param1', 'param2','param3', 'param4'})

    def test_get_source_invalid_var_type(self):
        with self.assertRaises(KeyError):
            self.cs.get_source('nonexistent_var_type')

    def test_get_source_invalid_belongs_to(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        with self.assertRaises(KeyError):
            self.cs.get_source('param1', belongs_to='invalid_action')

    def test_set_action_with_belongs_to(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        with self.assertRaises(KeyError):
            self.cs.set_categorical('optimizer2', choices=[dummy_action2], action=True, belongs_to='optimizer')

    def test_set_float_invalid_var_type_name(self):
        with self.assertRaises(KeyError):  # Corrected from ValueError to KeyError
            self.cs.set_float('learning-rate', low=0.001, high=0.01)
        with self.assertRaises(KeyError):  # Corrected from ValueError to KeyError
            self.cs.set_float('learning>rate', low=0.001, high=0.01)

    def test_set_int_invalid_belongs_to_name(self):
        with self.assertRaises(KeyError):
            self.cs.set_int('num_layers', low=1, high=5, belongs_to='invalid>action')

    def test_set_categorical_invalid_action_choice(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('optimizer', choices=[123], action=True)

    def test_set_float_invalid_range(self):
        with self.assertRaises(ValueError):
            self.cs.set_float('learning_rate', low=0.1, high=0.001)

    def test_get_var_invalid_var_type(self):
        with self.assertRaises(KeyError):
            self.cs.get_var('nonexistent_var_type')

    def test_get_var_invalid_belongs_to(self):
        with self.assertRaises(KeyError):
            self.cs.get_var('param1', belongs_to='nonexistent_action')

    def test_set_categorical_with_invalid_action_flag(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('optimizer', choices=[dummy_action1], action='not_bool')

    def test_set_float_with_invalid_step(self):
        with self.assertRaises(ValueError):
            self.cs.set_float('learning_rate', low=0.001, high=0.01, step='not_float')

    def test_set_int_with_invalid_step(self):
        with self.assertRaises(ValueError):
            self.cs.set_int('num_layers', low=1, high=5, step='not_int')

    def test_set_categorical_with_invalid_choices_type(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('activation', choices='not_a_list')

    def test_action_param_with_invalid_choices(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('optimizer', choices=[123, 'string'], action=True)

    def test_verify_missing_signature_parameters(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        # Missing signature parameters
        with self.assertRaises(ValueError):
            self.cs.verify_syntax(False)

    def test_sig_var_types_invalid_action_var_type(self):
        with self.assertRaises(KeyError):
            self.cs.sig_var_types('nonexistent_action')

    def test_get_var_missing_signature_parameters(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        with self.assertRaises(ValueError):
            self.cs.get_var('optimizer')

    def test_get_sources_with_invalid_input(self):
        with self.assertRaises(ValueError):
            self.cs.get_sources('not_a_list')

    def test_get_sources_with_non_tuple_elements(self):
        with self.assertRaises(KeyError):
            self.cs.get_sources(['not_a_tuple'])

    def test_set_float_with_invalid_log_type(self):
        with self.assertRaises(ValueError):
            self.cs.set_float('learning_rate', low=0.001, high=0.01, log='not_bool')

    def test_set_int_with_invalid_log_type(self):
        with self.assertRaises(ValueError):
            self.cs.set_int('num_layers', low=1, high=5, log='not_bool')

    def test_set_float_with_invalid_param_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_float(123, low=0.001, high=0.01)

    def test_set_int_with_invalid_param_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_int(123, low=1, high=5)

    def test_set_categorical_with_invalid_param_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical(123, choices=[1, 2, 3])

    def test_set_float_with_belongs_to_none_when_required(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        #Overlap is allowed between sig_var_types and the rest of the spaces
        #(It only needs to be prevented between action, non-action name spaces)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to=None)

    def test_set_float_with_belongs_to_not_none_when_not_required(self):
        with self.assertRaises(KeyError):
            self.cs.set_float('learning_rate', low=0.001, high=0.01, belongs_to='optimizer')

    def test_set_categorical_action_with_belongs_to_not_none(self):
        with self.assertRaises(KeyError):
            self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True, belongs_to='something')
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        #Should raise KeyError because 'some_action' is not a registered action
        with self.assertRaises(KeyError):
            self.cs.set_categorical('optimizer2', choices=['adam', 'sgd'], belongs_to='some_action')

    def test_get_var_with_invalid_belongs_to(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        with self.assertRaises(KeyError):
            self.cs.get_var('learning_rate', belongs_to='some_action')

    def test_get_action_vars_with_invalid_input(self):
        with self.assertRaises(KeyError):
            self.cs.get_action_vars(action_var_types='nonexistent_action')

    def test_get_non_action_vars_with_invalid_input(self):
        with self.assertRaises(KeyError):
            self.cs.get_non_action_vars(non_action_var_types='nonexistent_var')

    #****STILL UNDER CONSTRUCTION*****
    def test_overriding_action_parameter(self):
        #Add 2,6th params to 'optimizer_evens'
        self.cs.set_categorical('optimizer_evens', choices=[dummy_action_evens], action=True)
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer_evens')
        with self.assertRaises(KeyError): #since 1 is an odd number
            self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer_evens')
        self.cs.set_float('param6', low=0.0, high=1.0, belongs_to='optimizer_evens')
        self.assertEqual(
            self.cs.sig_var_types('optimizer_evens'),
            {'param2', 'param6'}
        )
        self.assertEqual(
            self.cs.sig_var_types(),
            {'param2', 'param6'}
        )

        #Override 'optimizer_evens', with dummy_action_odds.
        #Note that this will cause ALL the sig_var_types to be deleted, with none inherited resultantly.
        self.cs.set_categorical('optimizer_evens', choices=[dummy_action_odds], action=True)
        self.assertEqual(
            self.cs.sig_var_types('optimizer_evens'),
            set() #since none of the potential_sig_var_types(={'param1','param3','param5'}) were in the 'cs'.
        )
        self.assertEqual(
            self.cs.sig_var_types(),
            set()
        )

    def test_verify_objective(self):
        with self.assertRaises(ValueError):
            self.cs.verify_objective(warn_missing_var_types=False)
        with self.assertWarns(Warning):
            self.cs.verify_objective(warn_missing_var_types=True)
    """
    def test_sample_call_visitor(self):
        #Dedent the source code
        source = textwrap.dedent(source)
        #Parse the source code into an AST
        tree = ast.parse(source)
        #Create an instance of the visitor and walk the tree
        visitor = SampleCallVisitor()
        visitor.visit(tree)
    """

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

#Class: ActionSampler

In [81]:
class ActionSampler:
    def __init__(self, INIT_ACTION: ConfigSpace):
        if not isinstance(INIT_ACTION, ConfigSpace):
            raise ValueError(f"Input 'INIT_ACTION' must be an instance of ConfigSpace but found: {type(INIT_ACTION)}")

        self.action_vars = {var.var_type: var for var in INIT_ACTION.get_action_vars()}
        self.model_var_types = {action_var_type for action_var_type in self.action_vars.keys() if 'model' in action_var_type}

    def __len__(self):
        return len(self.action_vars)

    def __eq__(self, other: ActionSampler):
        return all([
            isinstance(other, ActionSampler),
            self.action_vars == other.action_vars,
            self.model_var_types == other.model_var_types
        ])

    def __str__(self):
        output = "**********ActionSampler**********\n"
        output += "".join([f"{action_var}\n" for action_var in self.action_vars.values()])
        return output

    def get_var(self, var_type: str, action_idx_or_action_func=None, sig_var_type: str=None) -> ActionVar:
        if var_type not in self.keys():
            raise KeyError(f"ActionSampler cannot retrieve Requested ActionVar: {var_type}, since it does NOT belong to: {self.keys()}")
        if action_idx_or_action_func != sig_var_type and (action_idx_or_action_func is None or sig_var_type is None):
            raise KeyError(f"Either both action_idx_or_action_func,sig_var_type should be None or both of them must be properly defined but found {(action_idx_or_action_func,sig_var_type)}")
        if sig_var_type is None:
            return self.action_vars[var_type]
        return self.get_action(var_type, action_idx_or_action_func).get_var(sig_var_type)

    def get_action(self, action_var_type: str, action_idx_or_action_func) -> Action:
        action_var = self.get_var(action_var_type)
        assert isinstance(action_var, ActionVar), f"type(action_var): {type(action_var)} != ActionVar"
        return action_var.get_action(action_idx_or_action_func)

    def is_fixed(self):
        return all(action_var.is_fixed() for action_var in self.action_vars.values())

    def keys(self):
        return self.action_vars.keys()

    def values(self):
        return self.action_vars.values()

    def items(self):
        return self.action_vars.items()

    def sample(self, action_var_type: str, trial: Trial):
        if trial is not None and not isinstance(trial, (optuna.trial.Trial, optuna.trial.FrozenTrial)):
            raise ValueError(f"Input 'trial' should be an object optuna::Trial or optuna::FrozenTrial but found: {type(trial)}")
        if trial is None:
            raise ValueError("Input 'trial' cannot be None when sampling is requested through ActionSampler.")

        action_var = self.get_var(action_var_type)
        assert isinstance(action_var, ActionVar), f"type(action_var): {type(action_var)} != ActionVar"

        sampled_action = action_var.sample(trial=trial)
        sampled_action.parameterize(trial=trial)

        return sampled_action

    def get_evaluated_params(self, source, exclude):
        if not isinstance(source, (Trial, FrozenTrial, dict)):
            raise ValueError(f"Input 'source' must be of type in (optuna::Trial, optuna::FrozenTrial, dict) but found: {source}")
        if not isinstance(exclude, (list, tuple, str)):
            raise ValueError(f"Input 'exclude' must be one of (list, tuple, str) but found type(exclude): {type(exclude)}")

        source = source if isinstance(source, dict) else source.params
        params = dict()

        if isinstance(exclude, (list, tuple)):
            exclude_set = set(exclude)
        else:
            exclude_set = {exclude}
        action_var_types = set(self.keys()) - exclude_set

        action_vars_to_assemble = {
            key: val for key, val in source.items()
            if key[0] in action_var_types and key[1] == 'None'
        }

        for key, val in action_vars_to_assemble.items():
            action_var = self.get_var(key[0])
            params[key] = action_var.get_loadable_action(val, source)

        return params

    def update(self, list_params: list, update_state_window: int, device: torch.device):
        #IF action_var_type == 'model_action'
        for action_var_type, action_var in self.items():
            relevant_params = [params for params in list_params if (action_var_type, 'None') in params]
            if not relevant_params:
                continue

            action_idxs = [params[(action_var_type,'None')] for params in relevant_params]
            action_var.update(vals=action_idxs)

            unique_action_idxs = list(dict.fromkeys(action_idxs).keys())

            if action_var_type in self.model_var_types:
                relevant_params = [
                    params|self.get_evaluated_params(source=params,exclude=[action_var_type])
                    for params in relevant_params
                ]

            action_var.update_actions(
                update_action_idxs=unique_action_idxs,
                list_params=relevant_params,
                update_state_window=update_state_window,
                device=device
            )

#Test: ActionSampler

In [41]:
#Assuming Var, Action, ModelAction, and ActionVar are defined as per the improved code.

#Fake Objective class
class Objective:
    def __init__(self, trial, combined_sampler, device, default_dict):
        self.trial = trial
        self.combined_sampler = combined_sampler
        self.device = device
        self.default_dict = default_dict

    def execute_objective(self):
        model = self.default_dict['model_action'](device=self.device)

        for param in model.parameters():
            param.data.fill_(0)

        model_container = Mock(spec=Learner)
        model_container.model = model

        return (0, model)

#Sample model classes
class ModelDefault(nn.Module):
    def __init__(self, rand_number: int):
        super().__init__()
        self.net = nn.Linear(16, 10)
        self.rand_number = torch.tensor(rand_number)

    def forward(self, x):
        return self.net(x) + self.rand_number

class ModelDropout(nn.Module):
    def __init__(self, activation: nn.Module, dropout_p: float):
        super().__init__()
        self.activation = activation
        self.dropout_p = dropout_p
        self.net = nn.Sequential(
            nn.Linear(16, 10),
            activation(),
            nn.Dropout(p=dropout_p),
            nn.Linear(10, 10)
        )

    def forward(self, x):
        return self.net(x)

#Action functions
def model_action_default(rand_number: int, device: torch.device):
    return ModelDefault(rand_number=rand_number).to(device)

def model_action_dropout(activation, dropout_p, device: torch.device):
    return ModelDropout(activation=activation, dropout_p=dropout_p).to(device)

def dls_action_default(batch_size: int, device: torch.device):
    return {'batch_size': batch_size, 'device': device}

def dls_action_normalize(batch_size: int, normalize: bool):
    return {'batch_size': batch_size, 'normalize': normalize}

def dls_action_activation(batch_size: int, normalize: bool, activation: nn.Module):
    return {'batch_size': batch_size, 'normalize': normalize, 'activation': activation}

class TestActionSampler(unittest.TestCase):
    def setUp(self):
        # Initialize UserConfigSpace
        self.cs = UserConfigSpace()

        # Define action hyperparameters
        self.cs.set_categorical(
            var_type='model_action',
            choices=[model_action_default, model_action_dropout],
            action=True
        )
        self.cs.set_categorical(
            var_type='dls_action',
            choices=[dls_action_default, dls_action_normalize, dls_action_activation],
            action=True
        )

        #Define signature parameters for model_action
        self.cs.set_int(
            var_type='rand_number',
            low=1,
            high=10,
            belongs_to='model_action'
        )
        self.cs.set_categorical(
            var_type='activation',
            choices=[nn.ReLU, nn.PReLU, nn.SiLU],
            belongs_to='model_action'
        )
        self.cs.set_float(
            var_type='dropout_p',
            low=0.0,
            high=1.0,
            belongs_to='model_action'
        )

        # Define signature parameters for dls_action
        self.cs.set_int(
            var_type='batch_size',
            low=16,
            high=64,
            belongs_to='dls_action'
        )
        self.cs.set_categorical(
            var_type='normalize',
            choices=[True, False],
            belongs_to='dls_action'
        )
        self.cs.set_categorical(
            var_type='activation',
            choices=[nn.ReLU, nn.SiLU],
            belongs_to='dls_action'
        )

        # Initialize ActionSampler with UserConfigSpace
        self.action_sampler = ActionSampler(self.cs)

    def test_initialization(self):
        #Test that ActionSampler initializes correctly
        self.assertEqual(len(self.action_sampler), 2)
        self.assertIn('model_action', self.action_sampler.keys())
        self.assertIn('dls_action', self.action_sampler.keys())
        self.assertIn('model_action', self.action_sampler.model_var_types)
        self.assertNotIn('dls_action', self.action_sampler.model_var_types)
    def test_initialization(self):
        # Test that ActionSampler initializes correctly
        self.assertEqual(len(self.action_sampler), 2)
        self.assertEqual(
            self.action_sampler.keys(),
            {'model_action', 'dls_action'}
        )
        self.assertEqual(
            self.action_sampler.model_var_types,
            {'model_action'}
        )

    def test_sample_actions(self):
        #Test sampling model_action
        trial = optuna.create_study(direction='minimize').ask()
        model_action = self.action_sampler.sample('model_action', trial)
        self.assertIsInstance(model_action, ModelAction)
        self.assertIsNotNone(model_action.params)
        self.assertIsNotNone(model_action._model_loader)

        #Test sampling dls_action
        trial = optuna.create_study(direction='minimize').ask()
        dls_action = self.action_sampler.sample('dls_action', trial)
        self.assertIsInstance(dls_action, Action)
        self.assertIsNotNone(dls_action.params)
        self.assertIsNotNone(dls_action._model_loader)

    def test_get_evaluated_params(self):
        #Create a source with both model_action and dls_action parameters
        source_params = {
            ('model_action', 'None'): 0,
            ('rand_number', 'model_action'): 5,
            ('dls_action', 'None'): 2,
            ('batch_size', 'dls_action'): 32,
            ('normalize', 'dls_action'): 1, #False
            ('activation', 'dls_action'): 0  # nn.ReLU
        }
        source_params_two = {
            ('model_action', 'None'): 1,
            ('activation', 'model_action'): 2, #nn.SiLU
            ('dropout_p', 'model_action'): 0.5,
            ('dls_action', 'None'): 1,
            ('batch_size', 'dls_action'): 48,
            ('normalize', 'dls_action'): 0, #True
        }
        list_params = [source_params, source_params_two]

        for count,source_params in enumerate(list_params):
            #Exclude 'model_action' and get evaluated params
            evaluated_params = self.action_sampler.get_evaluated_params(source_params, exclude='model_action')
            self.assertIn(('dls_action', 'None'), evaluated_params, msg=f"action_sampler.var_types: {self.action_sampler.keys()}")
            dls_action = evaluated_params[('dls_action', 'None')]
            self.assertIsInstance(dls_action, Action)
            self.assertNotIn(('model_action', 'None'), evaluated_params)
            if count==0:
                d = {'batch_size': 32, 'normalize': False, 'activation': nn.ReLU}
                self.assertEqual(
                    dls_action(device=None),
                    d,
                    msg=f"dls_action(): {dls_action(device=None)} != \nExpected: {d}"
                )
            else:
                d = {'batch_size': 48, 'normalize': True}
                self.assertEqual(
                    dls_action(device=None),
                    d,
                    msg=f"dls_action(): {dls_action(device=None)} != \nExpected: {d}"
                )

            #Exclude 'dls_action' and get evaluated params
            evaluated_params = self.action_sampler.get_evaluated_params(
                source_params,
                exclude='dls_action'
            )
            self.assertIn(('model_action', 'None'), evaluated_params)
            model_action = evaluated_params[('model_action', 'None')]
            self.assertIsInstance(model_action, ModelAction)
            self.assertNotIn(('dls_action', 'None'), evaluated_params)
            if count==0:
                d = {'rand_number': 5}
                self.assertEqual(
                    model_action._model_loader.keywords,
                    d,
                    msg=f"model_action._model_loader.keywords: {model_action._model_loader.keywords} != \nExpected: {d}"
                )
            else:
                d = {'dropout_p': 0.5, 'activation':nn.SiLU}
                self.assertEqual(
                    model_action._model_loader.keywords,
                    d,
                    msg = f"model_action._model_loader.keywords: {model_action._model_loader.keywords} != \nExpected: {d}"
                )

    def test_update_method(self):
        #Prepare list_params with mixed model_action and dls_action parameters
        list_params = [
            # Entry 1:
            {
                ('model_action', 'None'): 0,  # model_action_default
                ('rand_number', 'model_action'): 5,

                ('dls_action', 'None'): 1,  # dls_action_normalize
                ('batch_size', 'dls_action'): 32,
                ('normalize', 'dls_action'): 1,  # False

                ('lr', 'None'): 0.01  # non_action_var_type
            },
            # Entry 2:
            {
                ('model_action', 'None'): 1,  # model_action_dropout
                ('activation', 'model_action'): 1,  # nn.PReLU
                ('dropout_p', 'model_action'): 0.5,

                ('dls_action', 'None'): 2,  # dls_action_activation
                ('batch_size', 'dls_action'): 64,
                ('normalize', 'dls_action'): 1,  # False
                ('activation', 'dls_action'): 1,  # nn.SiLU

                ('lr', 'None'): 0.001  # non_action_var_type
            },
            # Entry 3:
            {
                ('model_action', 'None'): 1,  # model_action_dropout
                ('activation', 'model_action'): 0,  # nn.ReLU
                ('dropout_p', 'model_action'): 0.23,

                ('dls_action', 'None'): 0,  # dls_action_default
                ('batch_size', 'dls_action'): 32,

                ('lr', 'None'): 0.988  # non_action_var_type
            },
            # Entry 4:
            {
                ('dls_action', 'None'): 2,  # dls_action_activation
                ('batch_size', 'dls_action'): 64,
                ('normalize', 'dls_action'): 1,  # False
                ('activation', 'dls_action'): 0  # nn.ReLU
            },
            # Entry 5:
            {
                ('model_action', 'None'): 0,  # model_action_default
                ('rand_number', 'model_action'): 8,
                ('lr', 'None'): 0.02
            }
        ]

        #Call update with update_state_window=1
        self.action_sampler.update(list_params, update_state_window=1, device=torch.device('cpu'))

        #Verify that the actions have been updated correctly
        #For model_action
        model_action_var = self.action_sampler.get_var('model_action')
        self.assertEqual(model_action_var.distribution(indices=True), [0,0,1,1])

        #For dls_action
        dls_action_var = self.action_sampler.get_var('dls_action')
        self.assertEqual(dls_action_var.distribution(indices=True), [0,1,2,2])

        #Determine which model action should have updated state
        #The unique action indices in the order they appear are [0, 1]
        #Since update_state_window=1, only the first action (index 0) should have its state updated

        model_action_default = model_action_var.get_action(0)
        model_action_dropout = model_action_var.get_action(1)

        #Verify that only the first model action has updated action_state
        self.assertIsNotNone(model_action_default.action_state)
        self.assertIsNone(model_action_dropout.action_state)

        #Verify that dls actions do not have action_state updated
        for idx in [0, 1, 2]:
            dls_action = dls_action_var.get_action(idx)
            self.assertIsInstance(dls_action, Action)
            self.assertIsNone(dls_action.action_state)

        #Additionally, we can verify the updated variables
        #For model_action_default
        rand_number_var = model_action_var.get_var(0, 'rand_number')
        self.assertEqual(rand_number_var.distribution(), {'low':5, 'high':8, 'step':1, 'log':False})

        #For model_action_dropout
        activation_var = model_action_var.get_var(1, 'activation')
        self.assertEqual(activation_var.distribution(), [nn.ReLU, nn.PReLU])

        dropout_p_var = model_action_var.get_var(1, 'dropout_p')
        self.assertEqual(dropout_p_var._dist['low'], 0.23)
        self.assertEqual(dropout_p_var._dist['high'], 0.5)

        for i in range(0,3):
            batch_size_var = dls_action_var.get_var(i, 'batch_size')
            self.assertEqual(batch_size_var._dist['low'], 32)
            self.assertEqual(batch_size_var._dist['high'], 64)

        for i in range(1,3):
            normalize_var = dls_action_var.get_var(i, 'normalize')
            self.assertEqual(normalize_var.distribution(), [False])

        for i in range(2,3):
            activation_var = dls_action_var.get_var(i, 'activation')
            self.assertEqual(activation_var.distribution(), [nn.ReLU, nn.SiLU])

    def test_update_dls_action(self):
        #Prepare list_params with only dls_action parameters
        list_params = [
            # Entry 1: dls_action_default, batch_size=32
            {
                ('dls_action', 'None'): 0,
                ('batch_size', 'dls_action'): 32
            },
            # Entry 2: dls_action_normalize, batch_size=48, normalize=True
            {
                ('dls_action', 'None'): 1,
                ('batch_size', 'dls_action'): 48,
                ('normalize', 'dls_action'): 0  # True
            },
            # Entry 3: dls_action_activation, batch_size=64, normalize=False, activation=nn.SiLU
            {
                ('dls_action', 'None'): 2,
                ('batch_size', 'dls_action'): 64,
                ('normalize', 'dls_action'): 1,  # False
                ('activation', 'dls_action'): 1  # nn.SiLU
            }
        ]

        #Call update with update_state_window=1 (should be ignored for dls_action)
        self.action_sampler.update(list_params, update_state_window=1, device=torch.device('cpu'))

        #Verify that the dls actions have updated configuration spaces
        dls_action_var = self.action_sampler.get_var('dls_action')

        batch_size_var = dls_action_var.get_var(0, 'batch_size')
        self.assertEqual(batch_size_var._dist['low'], 32)
        self.assertEqual(batch_size_var._dist['high'], 64)

        normalize_var = dls_action_var.get_var(1, 'normalize')
        self.assertEqual(normalize_var.distribution(), [True, False])

        activation_var = dls_action_var.get_var(2, 'activation')
        self.assertEqual(activation_var.distribution(), [nn.SiLU])

        # Verify that action_state is not updated for dls actions
        for idx in [0, 1, 2]:
            action = dls_action_var.get_action(idx)
            self.assertIsNone(action.action_state)

    def test_update_model_action(self):
        # Prepare list_params with only model_action parameters
        list_params = [
            # Entry 1: model_action_default, rand_number=5
            {
                ('model_action', 'None'): 0,
                ('rand_number', 'model_action'): 5,
                ('lr', 'None'): 0.01
            },
            # Entry 2: model_action_dropout, activation=nn.SiLU, dropout_p=0.3
            {
                ('model_action', 'None'): 1,
                ('activation', 'model_action'): 2,  # nn.SiLU
                ('dropout_p', 'model_action'): 0.3,
                ('lr', 'None'): 0.001
            },
            # Entry 3: model_action_default, rand_number=8
            {
                ('model_action', 'None'): 0,
                ('rand_number', 'model_action'): 8,
                ('lr', 'None'): 0.02
            }
        ]

        # Call update with update_state_window=2
        self.action_sampler.update(list_params, update_state_window=2, device=torch.device('cpu'))

        # Verify that the model actions have updated configuration spaces
        model_action_var = self.action_sampler.get_var('model_action')

        rand_number_var = model_action_var.get_var(0, 'rand_number')
        self.assertEqual(rand_number_var.distribution(), {'low': 5, 'high': 8, 'log': False, 'step': 1})

        activation_var = model_action_var.get_var(1, 'activation')
        self.assertEqual(activation_var.distribution(), [nn.SiLU])

        dropout_p_var = model_action_var.get_var(1, 'dropout_p')
        self.assertEqual(dropout_p_var._dist['low'], 0.3)
        self.assertEqual(dropout_p_var._dist['high'], 0.3)

        # Verify that action_state is updated for model actions
        model_action_default = model_action_var.get_action(0)
        model_action_dropout = model_action_var.get_action(1)

        self.assertIsNotNone(model_action_default.action_state)
        self.assertIsNotNone(model_action_dropout.action_state)

    def test_update_with_mixed_actions(self):
        # Prepare list_params with both model_action and dls_action parameters
        list_params = [
            # Entry 1: model_action_default, rand_number=5
            {
                ('model_action', 'None'): 0,
                ('rand_number', 'model_action'): 5,
                ('lr', 'None'): 0.01,
                ('dls_action', 'None'): 1,
                ('batch_size', 'dls_action'): 48,
                ('normalize', 'dls_action'): 0  # True
            },
            # Entry 2: model_action_dropout, activation=nn.SiLU, dropout_p=0.3
            {
                ('model_action', 'None'): 1,
                ('activation', 'model_action'): 2,  # nn.SiLU
                ('dropout_p', 'model_action'): 0.3,
                ('lr', 'None'): 0.001,
                ('dls_action', 'None'): 2,
                ('batch_size', 'dls_action'): 64,
                ('normalize', 'dls_action'): 1,  # False
                ('activation', 'dls_action'): 1  # nn.SiLU
            }
        ]

        # Call update with update_state_window=1
        self.action_sampler.update(list_params, update_state_window=1, device=torch.device('cpu'))

        # Verify that the model actions have updated states
        model_action_var = self.action_sampler.get_var('model_action')
        model_action_default = model_action_var.get_action(0)
        model_action_dropout = model_action_var.get_action(1)

        self.assertIsNotNone(model_action_default.action_state)
        self.assertIsNone(model_action_dropout.action_state)  # Since update_state_window=1

        # Verify that dls actions do not have action_state updated
        dls_action_var = self.action_sampler.get_var('dls_action')
        for idx in [0, 1, 2]:
            action = dls_action_var.get_action(idx)
            self.assertIsNone(action.action_state)

        #Verify that get_evaluated_params correctly assembles actions
        #When updating model_action, dls_action should be prepared
        #This is tested by checking if the dls_action is included in the parameters passed to Objective
        #(In a real implementation, the Objective function would use these actions)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

<ipython-input-36-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('rand_number', 'model_action'), ('activation', 'model_action')]

  warnings.warn(msg)
<ipython-input-36-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('rand_number', 'model_action'), ('activation', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action'), ('activation', 'dls_action'), ('normalize', 'dls_action')]

  warnings.warn(msg)
<ipython-input-36-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('activation', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action'), ('activation', 'dls_action'), ('normalize', 'dls_action')]

 

#Class: PrimitiveSampler

In [96]:
class PrimitiveSampler:
    def __init__(self, INIT_PRIM: ConfigSpace):
        if not isinstance(INIT_PRIM, ConfigSpace):
            raise ValueError(f"Input 'INIT_PRIM' must be an instance of ConfigSpace but found type(INIT_PRIM): {type(INIT_PRIM)}")
        self.vars = {var.var_type: var for var in INIT_PRIM.get_non_action_vars()}

    def __len__(self):
        return len(self.vars)

    def __eq__(self, other: PrimitiveSampler):
        return isinstance(other, PrimitiveSampler) and self.vars == other.vars

    def __str__(self):
        output = "**********PrimitiveSampler**********\n"
        output += "".join([f"{var}\n" for var in self.vars.values()])
        return output

    def get_var(self, var_type):
        if var_type not in self.keys():
            raise KeyError(f"Requested 'var_type': {var_type} is not in PrimitiveSampler.keys(): {self.keys()}.")
        return self.vars[var_type]

    def keys(self):
        return self.vars.keys()

    def values(self):
        return self.vars.values()

    def items(self):
        return self.vars.items()

    def is_fixed(self):
        return all(var.is_fixed() for var in self.values())

    def get_evaluated_params(self, source):
        if not isinstance(source, (Trial, FrozenTrial, dict)):
            raise ValueError(f"Input 'source' must be of type in (optuna::Trial, optuna::FrozenTrial, dict) but found: {source}")

        source = source if isinstance(source, dict) else source.params

        params = {
            (var_type,belongs_to):val for (var_type,belongs_to),val in source.items()
            if var_type in self.keys() and belongs_to=='None'
        }

        for key,val in params.items():
            var = self.get_var(key[0])
            if var.contain_idx(val):
                val = var.convert_idx_to_val(val)
            params[key] = val

        return params

    def sample(self, var_type: str, trial):
        if trial is not None and not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' should be an instance of optuna.Trial or optuna.FrozenTrial but found: {trial}")
        if trial is None:
            raise ValueError("Input 'trial' cannot be 'None'. Please provide an optuna.Trial or optuna.FrozenTrial instance.")
        if var_type not in self.keys():
            raise KeyError(f"Requested var_type: {var_type} does not exist in the PrimitiveSampler.")

        #sample value from the variable of type 'var_type'.
        sampled_val = self.get_var(var_type).sample(trial)

        return sampled_val

    def update_vars(self, list_params: list):
        #Update variables based on the list of parameters
        for var_type, var in self.items():
            var.update(vals=[params[(var_type, 'None')] for params in list_params if (var_type, 'None') in params.keys()])

#Class: CombinedSampler

In [90]:
class CombinedSampler:
    def __init__(self, INIT_CONFIG: ConfigSpace):
        if not isinstance(INIT_CONFIG, ConfigSpace):
            raise ValueError(f"Input 'INIT_CONFIG' must be of type in ConfigSpace but found type(INIT_CONFIG): {type(INIT_CONFIG)}")
        self.action_sampler = ActionSampler(INIT_CONFIG)
        self.primitive_sampler = PrimitiveSampler(INIT_CONFIG)
        self._sig_var_types = INIT_CONFIG.sig_var_types()

    def __str__(self):
        output = "CombinedSampler:\n"
        output += f"{self.action_sampler}\n"
        return output + f"{self.primitive_sampler}\n"

    def is_fixed(self):
        return self.primitive_sampler.is_fixed() and self.action_sampler.is_fixed()

    def get_var(self, var_type: str, *args, **kwargs) -> Var:
        if var_type in self.non_action_var_types():
            return self.primitive_sampler.get_var(var_type)
        return self.action_sampler.get_var(var_type, *args, **kwargs)

    def get_action(self, action_var_type: str, action_idx_or_action_func) -> Action:
        return self.action_sampler.get_action(action_var_type, action_idx_or_action_func)

    def non_action_var_types(self):
        return set(self.primitive_sampler.keys())

    def action_var_types(self):
        return set(self.action_sampler.keys())

    def model_action_var_types(self):
        return self.action_sampler.model_var_types

    def sig_var_types(self, action_var_type: str=None, action_idx_or_action_func=None):
        if action_var_type is None and action_idx_or_action_func is None:
            return self._sig_var_types
        elif action_var_type is None or action_idx_or_action_func is None:
            raise KeyError("Either 'action_var_type' and 'action_idx_or_action_func' must be provided or both must be None.")
        return set(self.get_action(action_var_type, action_idx_or_action_func).keys())

    def var_types(self):
        return set(self.primitive_sampler.keys()) | set(self.action_sampler.keys())

    def contain_idx(self, *args, idx):
        return self.get_var(*args).contain_idx(idx)

    def convert_idx_to_val(self, *args, idx):
        return self.get_var(*args).convert_idx_to_val(idx)

    def contain_val(self, *args, val):
        return self.get_var(*args).contain_val(val)

    def convert_val_to_idx(self, *args, val):
        return self.get_var(*args).convert_val_to_idx(val)

    def sample(self, var_type: str, trial: Trial, default_val = None):
        """
        For proper sampling, 'trial' must be an instance of optuna.Trial or optuna.FrozenTrial.
        If 'trial' is None, then 'default_val' will be returned.
        """
        if var_type in self._sig_var_types:
            raise KeyError(f"Requested var_type: {var_type} is a signature var_type, which is not allowed to be sampled directly.")
        if var_type not in self.var_types():
            raise KeyError(f"Requested var_type: {var_type} does not exist in the CombinedSampler.")
        var = self.get_var(var_type)
        valid_trial = trial is not None and isinstance(trial, (Trial, FrozenTrial))

        #Any 'default_val' must be in the current distribution of the var_type
        if not valid_trial and not var.contain_val(default_val):
            raise ValueError(f"Var of type: {var_type} cannot produce the given 'default_val': {default_val} since its current distribution is: {var.distribution()}")
        if not valid_trial:
            #Return 'default_val' if 'trial' is invalid but 'default_val' is valid
            return default_val

        sample_from = self.action_sampler if var_type in self.action_var_types() else self.primitive_sampler
        return sample_from.sample(var_type, trial)

    def update(self, top_k_trials: list, update_state_window: int, device: torch.device):
        #Empty list for top_k_trials is not allowed. (Auto-HPO should have been finished before any empty-update is justifiable).
        if not len(top_k_trials):
            raise ValueError("Input 'top_k_trials' cannot be empty")

        #Convert trial params to the required format
        list_params = [{tuple(key.split('>')): val for key, val in trial.params.items()} for trial in top_k_trials]

        #Update primitive_sampler
        self.primitive_sampler.update_vars(list_params=list_params)

        #Update list_params by evaluating non-action sampled idx(s) to value(s).
        list_params = [params|self.primitive_sampler.get_evaluated_params(source=params) for params in list_params]

        #Update action_sampler
        self.action_sampler.update(
            list_params=list_params,
            update_state_window=update_state_window,
            device=device
        )

#Test: CombinedSampler



In [44]:
#Assuming Var, Action, ModelAction, and ActionVar are defined and imported
#Also assuming that CombinedSampler class is defined as per the improved code

#Fake Objective class
class Objective:
    def __init__(self, trial, combined_sampler, device, default_dict):
        self.trial = trial
        self.combined_sampler = combined_sampler
        self.device = device
        self.default_dict = default_dict

    def execute_objective(self):
        model = self.default_dict['model_action'](device=self.device)

        for param in model.parameters():
            param.data.fill_(0)

        model_container = Mock(spec=Learner)
        model_container.model = model

        return (0, model)

#Sample model classes
class ModelDefault(nn.Module):
    def __init__(self, rand_number: int):
        super().__init__()
        self.net = nn.Linear(16, 10)
        self.rand_number = torch.tensor(rand_number)

    def forward(self, x):
        return self.net(x) + self.rand_number

class ModelDropout(nn.Module):
    def __init__(self, activation: nn.Module, dropout_p: float):
        super().__init__()
        self.activation = activation
        self.dropout_p = dropout_p
        self.net = nn.Sequential(
            nn.Linear(16, 10),
            activation(),
            nn.Dropout(p=dropout_p),
            nn.Linear(10, 10)
        )

    def forward(self, x):
        return self.net(x)

class ModelDropoutVariant(nn.Module):
    def __init__(self, rand_number: int, activation: nn.Module, dropout_p: int):
        super().__init__()
        self.rand_number = rand_number
        self.net = nn.Sequential(
            nn.Linear(in_features=16, out_features=10),
            activation(),
            nn.Dropout(p=dropout_p),
            nn.Linear(in_features=10, out_features=10)
        )

    def forward(self, x):
        return self.net(x)

# Action functions
def model_action_default(rand_number: int, device: torch.device):
    return ModelDefault(rand_number=rand_number).to(device)

def model_action_dropout(activation, dropout_p, device: torch.device):
    return ModelDropout(activation=activation, dropout_p=dropout_p).to(device)

def model_action_dropout_variant(rand_number: int, activation, dropout_p, device: torch.device):
    return ModelDropoutVariant(rand_number=rand_number, activation=activation, dropout_p=dropout_p).to(device)

def dls_action_default(batch_size: int, device: torch.device):
    return {'batch_size': batch_size, 'device': device}

def dls_action_normalize(batch_size: int, normalize: bool):
    return {'batch_size': batch_size, 'normalize': normalize}

def dls_action_activation(batch_size: int, normalize: bool, activation: nn.Module):
    return {'batch_size': batch_size, 'normalize': normalize, 'activation': activation}

class TestCombinedSampler(unittest.TestCase):
    def setUp(self):
        # Initialize UserConfigSpace
        self.cs = UserConfigSpace()

        # Define action hyperparameters
        self.cs.set_categorical(
            var_type='model_action',
            choices=[model_action_default, model_action_dropout, model_action_dropout_variant],
            action=True
        )
        self.cs.set_categorical(
            var_type='dls_action',
            choices=[dls_action_default, dls_action_normalize, dls_action_activation],
            action=True
        )

        # Define signature parameters for 'model_action'
        self.cs.set_categorical(
            var_type='rand_number',
            choices=list(range(1, 11)),
            belongs_to='model_action'
        )
        self.cs.set_categorical(
            var_type='activation',
            choices=[nn.ReLU, nn.PReLU, nn.SiLU],
            belongs_to='model_action'
        )
        self.cs.set_float(
            var_type='dropout_p',
            low=0.0,
            high=1.0,
            belongs_to='model_action'
        )

        # Define signature parameters for 'dls_action'
        self.cs.set_int(
            var_type='batch_size',
            low=16,
            high=64,
            belongs_to='dls_action'
        )
        self.cs.set_categorical(
            var_type='normalize',
            choices=[True, False],
            belongs_to='dls_action'
        )
        self.cs.set_categorical(
            var_type='activation',
            choices=[nn.ReLU, nn.SiLU],
            belongs_to='dls_action'
        )

        # Define non-action variables
        self.cs.set_float(
            var_type='lr',
            low=0.0001,
            high=0.01,
            log=True
        )
        self.cs.set_float(
            var_type='wd',
            low=0.0001,
            high=0.1,
            log=True
        )

        # Initialize CombinedSampler with UserConfigSpace
        self.combined_sampler = CombinedSampler(self.cs)

    def create_mock_trial(self, params):
        trial = Mock(spec=optuna.trial.FrozenTrial)
        trial.params = params
        return trial

    def test_initialization(self):
        #Test that CombinedSampler initializes correctly
        self.assertIsInstance(self.combined_sampler.action_sampler, ActionSampler)
        self.assertIsInstance(self.combined_sampler.primitive_sampler, PrimitiveSampler)

        #Check var_types retrieval/categorization.
        self.assertEqual(self.combined_sampler.action_var_types(), {'model_action', 'dls_action'})
        self.assertEqual(self.combined_sampler.non_action_var_types(), {'lr', 'wd'})
        self.assertEqual(self.combined_sampler.sig_var_types(), {'rand_number', 'batch_size', 'normalize', 'activation', 'dropout_p'})
        self.assertEqual(self.combined_sampler.var_types(), {'model_action', 'dls_action', 'lr', 'wd'})

    def test_sample_non_action_vars(self):
        trial = optuna.create_study(direction='minimize').ask()
        lr = self.combined_sampler.sample('lr', trial)
        wd = self.combined_sampler.sample('wd', trial)
        #Assert that the sampled values are within the expected ranges
        self.assertTrue(0.0001 <= lr <= 0.01)
        self.assertTrue(0.0001 <= wd <= 0.1)

    def test_sample_model_actions(self):
        trial = optuna.create_study(direction='minimize').ask()
        model_action = self.combined_sampler.sample('model_action', trial)
        self.assertIsInstance(model_action, ModelAction)
        self.assertIsNotNone(model_action.params)
        self.assertIsNotNone(model_action._model_loader)
        self.assertEqual(
            model_action.params,
            model_action._model_loader.keywords,
            msg=f"model_action.params: {model_action.params} != model_action._model_loader.keywores: {model_action._model_loader.keywords }"
        )
        #Now check which action_func was sampled and assert accordingly
        if model_action.action_func == model_action_default:
            rand_number = model_action.params['rand_number']
            self.assertIn(rand_number, list(range(1, 11)))
        elif model_action.action_func == model_action_dropout:
            activation = model_action.params['activation']
            dropout_p = model_action.params['dropout_p']
            self.assertIn(activation, [nn.ReLU, nn.PReLU, nn.SiLU])
            self.assertTrue(0.0 <= dropout_p <= 1.0)
        elif model_action.action_func == model_action_dropout_variant:
            rand_number = model_action.params['rand_number']
            activation = model_action.params['activation']
            dropout_p = model_action.params['dropout_p']
            self.assertIn(rand_number, list(range(1, 11)))
            self.assertIn(activation, [nn.ReLU, nn.PReLU, nn.SiLU])
            self.assertTrue(0.0 <= dropout_p <= 1.0)
        else:
            raise ValueError(f"Unexpected model_action action_func: {model_action.action_func}")

    def test_sample_dls_actions(self):
        trial = optuna.create_study(direction='minimize').ask()
        dls_action = self.combined_sampler.sample('dls_action', trial)
        self.assertIsInstance(dls_action, Action)
        self.assertFalse(isinstance(dls_action, ModelAction))
        self.assertIsNotNone(dls_action.params)
        self.assertIsNotNone(dls_action._model_loader)
        self.assertEqual(
            dls_action.params,
            dls_action._model_loader.keywords
        )
        #Now check which action_func was sampled and assert accordingly
        if dls_action.action_func == dls_action_default:
            batch_size = dls_action.params['batch_size']
            self.assertTrue(16 <= batch_size <= 64)
        elif dls_action.action_func == dls_action_normalize:
            batch_size = dls_action.params['batch_size']
            normalize = dls_action.params['normalize']
            self.assertTrue(16 <= batch_size <= 64)
            self.assertIn(normalize, [True, False])
        elif dls_action.action_func == dls_action_activation:
            batch_size = dls_action.params['batch_size']
            normalize = dls_action.params['normalize']
            activation = dls_action.params['activation']
            self.assertTrue(16 <= batch_size <= 64)
            self.assertIn(normalize, [True, False])
            self.assertIn(activation, [nn.ReLU, nn.SiLU])
        else:
            raise ValueError(f"Unexpected dls_action action_func: {dls_action.action_func}")

    def test_update(self):
        #Prepare mock trials with various combinations
        trial1_params = {
            'model_action>None': 0,  #model_action_default
            'rand_number>model_action': 5, #6

            'dls_action>None': 1,  #dls_action_normalize
            'batch_size>dls_action': 32,
            'normalize>dls_action': 0,  #True

            'lr>None': 0.005,
            'wd>None': 0.001
        }
        trial2_params = {
            'model_action>None': 1,  #model_action_dropout
            'activation>model_action': 2,  #nn.SiLU
            'dropout_p>model_action': 0.3,

            'dls_action>None': 2,  #dls_action_activation
            'batch_size>dls_action': 48,
            'normalize>dls_action': 1,  #False
            'activation>dls_action': 1,  #nn.SiLU

            'lr>None': 0.002,
            'wd>None': 0.01
        }
        trial3_params = {
            'model_action>None': 2,  #model_action_dropout_variant
            'rand_number>model_action': 7, #8
            'activation>model_action': 0,  #nn.ReLU
            'dropout_p>model_action': 0.5,

            'dls_action>None': 0,  #dls_action_default
            'batch_size>dls_action': 64,

            'lr>None': 0.001,
            'wd>None': 0.005
        }
        #Create trials
        trial1 = self.create_mock_trial(trial1_params)
        trial2 = self.create_mock_trial(trial2_params)
        trial3 = self.create_mock_trial(trial3_params)
        top_k_trials = [trial1, trial2, trial3]
        #Call update with update_state_window=2
        self.combined_sampler.update(top_k_trials=top_k_trials, update_state_window=2, device=torch.device('cpu'))
        # Now verify that the variables and actions have been updated correctly
        # For non-action variables
        lr_var = self.combined_sampler.get_var('lr')
        wd_var = self.combined_sampler.get_var('wd')
        #Note that it reflects ALL the 'k' number of params in top_k_trials
        self.assertEqual(lr_var._dist['low'], 0.001)
        self.assertEqual(lr_var._dist['high'], 0.005)
        self.assertEqual(wd_var._dist['low'], 0.001)
        self.assertEqual(wd_var._dist['high'], 0.01)
        #For model_action
        model_action_var = self.combined_sampler.action_sampler.get_var('model_action')
        self.assertEqual(model_action_var.distribution(indices=True), [0, 1, 2])
        #Check action_states
        model_action_default = model_action_var.get_action(0)
        model_action_dropout = model_action_var.get_action(1)
        model_action_dropout_variant = model_action_var.get_action(2)
        self.assertIsNotNone(model_action_default.action_state)
        self.assertIsNotNone(model_action_dropout.action_state)
        self.assertIsNone(model_action_dropout_variant.action_state)
        #For dls_action(action_state should not be updated)
        dls_action_var = self.combined_sampler.action_sampler.get_var('dls_action')
        for idx in [0, 1, 2]:
            action = dls_action_var.get_action(idx)
            self.assertIsNone(action.action_state)

    def test_sampling_after_update(self):
        #Prepare and update as before
        self.test_update()
        self.assertEqual(self.combined_sampler.get_var('model_action').distribution(True), [0,1,2])
        self.assertEqual(self.combined_sampler.get_var('dls_action').distribution(True), [0,1,2])
        #After updating, sample variables and check that distributions are updated
        trial = optuna.create_study(direction='minimize').ask()
        for var_type in self.combined_sampler.var_types():
            val = self.combined_sampler.sample(var_type, trial)
            if var_type=='model_action':
                self.assertIsInstance(val, ModelAction)
                self.assertIn(val.action_func, [model_action_default, model_action_dropout, model_action_dropout_variant])
                self.assertEqual(val(device=None)(torch.randn(16,16)).shape, torch.Size([16,10]))
            elif var_type=='dls_action':
                self.assertIsInstance(val, Action)
                self.assertIn(val.action_func, [dls_action_default, dls_action_normalize, dls_action_activation])
            elif var_type=='lr':
                self.assertTrue(0.001 <= val <= 0.005)
            elif var_type=='wd':
                self.assertTrue(0.001 <= val <= 0.01)

    def test_get_evaluated_params(self):
        #Prepare a source with both action and non-action parameters
        source_params = {
            ('model_action', 'None'): 2,  # model_action_dropout_variant
            ('rand_number', 'model_action'): 7, #8
            ('activation', 'model_action'): 1,  # nn.PReLU
            ('dropout_p', 'model_action'): 0.4,

            ('dls_action', 'None'): 2,  # dls_action_activation
            ('batch_size', 'dls_action'): 48,
            ('normalize', 'dls_action'): 1,  # False
            ('activation', 'dls_action'): 0,  # nn.ReLU

            ('lr', 'None'): 0.001,
            ('wd', 'None'): 0.005
        }
        #Get evaluated params from primitive_sampler
        evaluated_params = self.combined_sampler.primitive_sampler.get_evaluated_params(source_params)
        expected_params = {
            ('lr', 'None'): 0.001,
            ('wd', 'None'): 0.005
        }
        self.assertEqual(evaluated_params, expected_params)
        #Get evaluated actions from action_sampler
        evaluated_actions = self.combined_sampler.action_sampler.get_evaluated_params(source_params, exclude=[])
        #Check that both model_action and dls_action are in evaluated_actions
        self.assertIn(('model_action', 'None'), evaluated_actions)
        self.assertIn(('dls_action', 'None'), evaluated_actions)
        model_action = evaluated_actions[('model_action', 'None')]
        dls_action = evaluated_actions[('dls_action', 'None')]
        self.assertIsInstance(model_action, ModelAction)
        self.assertIsInstance(dls_action, Action)
        #Check model_action._model_loader parameters
        model_action_loader_params = model_action._model_loader.keywords
        self.assertEqual(model_action_loader_params['rand_number'], 8)
        self.assertEqual(
            model_action_loader_params['activation'],
            nn.PReLU,
            msg=f"model_action: {model_action.action_func}, model_action_loader_params: {model_action_loader_params}")
        self.assertEqual(model_action_loader_params['dropout_p'], 0.4)
        #Check dls_action parameters
        dls_model_loader_params = dls_action._model_loader.keywords
        self.assertEqual(dls_model_loader_params['batch_size'], 48)
        self.assertEqual(dls_model_loader_params['normalize'], False)
        self.assertEqual(
            dls_model_loader_params['activation'],
            nn.ReLU,
            msg=f"dls_action: {dls_action.action_func}, dls_model_loader_params: {dls_model_loader_params}")

    def test_combination_of_actions(self):
        #Test sampling multiple times to cover combinations
        for _ in range(10):
            trial = optuna.create_study(direction='minimize').ask()
            model_action = self.combined_sampler.sample('model_action', trial)
            dls_action = self.combined_sampler.sample('dls_action', trial)
            lr = self.combined_sampler.sample('lr', trial)
            wd = self.combined_sampler.sample('wd', trial)

            #Check types
            self.assertIsInstance(model_action, ModelAction)
            self.assertIsInstance(dls_action, Action)
            self.assertTrue(0.0001 <= lr <= 0.01)
            self.assertTrue(0.0001 <= wd <= 0.1)

            #Check model_action parameters
            if model_action.action_func == model_action_default:
                rand_number = model_action.params['rand_number']
                self.assertIn(rand_number, list(range(1, 11)))
            elif model_action.action_func == model_action_dropout:
                activation = model_action.params['activation']
                dropout_p = model_action.params['dropout_p']
                self.assertIn(activation, [nn.ReLU, nn.PReLU, nn.SiLU])
                self.assertTrue(0.0 <= dropout_p <= 1.0)
            elif model_action.action_func == model_action_dropout_variant:
                rand_number = model_action.params['rand_number']
                activation = model_action.params['activation']
                dropout_p = model_action.params['dropout_p']
                self.assertIn(rand_number, list(range(1, 11)))
                self.assertIn(activation, [nn.ReLU, nn.PReLU, nn.SiLU])
                self.assertTrue(0.0 <= dropout_p <= 1.0)
            else:
                raise ValueError(f"Unexpected model_action action_func: {model_action.action_func}")

            #Check dls_action parameters similarly
            if dls_action.action_func == dls_action_default:
                batch_size = dls_action.params['batch_size']
                self.assertTrue(16 <= batch_size <= 64)
            elif dls_action.action_func == dls_action_normalize:
                batch_size = dls_action.params['batch_size']
                normalize = dls_action.params['normalize']
                self.assertTrue(16 <= batch_size <= 64)
                self.assertIn(normalize, [True, False])
            elif dls_action.action_func == dls_action_activation:
                batch_size = dls_action.params['batch_size']
                normalize = dls_action.params['normalize']
                activation = dls_action.params['activation']
                self.assertTrue(16 <= batch_size <= 64)
                self.assertIn(normalize, [True, False])
                self.assertIn(activation, [nn.ReLU, nn.SiLU])
            else:
                raise ValueError(f"Unexpected dls_action action_func: {dls_action.action_func}")

    def test_invalid_variable_type(self):
        trial = optuna.create_study(direction='minimize').ask()
        with self.assertRaises(KeyError):
            self.combined_sampler.sample('invalid_var_type', trial)

    def test_invalid_default_value(self):
        with self.assertRaises(ValueError):
            self.combined_sampler.sample(
                var_type='lr',
                trial=None,
                #Throw in a value out of range
                default_val=self.combined_sampler.get_var('lr').distribution()['high']+10.0
            )

    def test_update_with_no_trials(self):
        with self.assertRaises(ValueError):
            self.combined_sampler.update(top_k_trials=[], update_state_window=1, device=torch.device('cpu'))

    def test_update_with_large_window(self):
        #Prepare one trial
        trial_params = {
            'model_action>None': 0,  #model_action_default
            'rand_number>model_action': 5, #6
            'lr>None': 1e-4,
            'wd>None': 1e-3
        }
        trial = self.create_mock_trial(trial_params)
        #Call update with update_state_window larger than number of trials
        #Ensure that update_state_window does not cause errors
        self.combined_sampler.update(top_k_trials=[trial], update_state_window=5, device=torch.device('cpu'))

        #Check that the (primitive)variables are updated based on the trial
        lr_var = self.combined_sampler.get_var('lr')
        self.assertEqual(lr_var._dist['low'], 1e-4)
        self.assertEqual(lr_var._dist['high'], 1e-4)
        wd_var = self.combined_sampler.get_var('wd')
        self.assertEqual(wd_var._dist['low'], 1e-3)
        self.assertEqual(wd_var._dist['high'], 1e-3)

        #Check that the ModelAction(and its signature Vars) are updated properly
        model_action_var = self.combined_sampler.action_sampler.get_var('model_action')
        self.assertEqual(model_action_var.distribution(indices=True), [0])
        rand_number_var = model_action_var.get_var(model_action_default,'rand_number')
        self.assertEqual(rand_number_var.distribution(), [6])

    def test_missing_action_variable_in_trial(self):
        trial_params = {
            'lr>None': 0.005,
            'wd>None': 0.001
            # 'model_action-None' is missing
        }
        trial = self.create_mock_trial(trial_params)

        #Call update and expect that it handles missing action vars
        self.combined_sampler.update(top_k_trials=[trial], update_state_window=2, device=torch.device('cpu'))

        #Check that non-action vars are updated
        lr_var = self.combined_sampler.get_var('lr')
        self.assertEqual(lr_var._dist['low'], 0.005)
        self.assertEqual(lr_var._dist['high'], 0.005)
        wd_var = self.combined_sampler.get_var('wd')
        self.assertEqual(wd_var._dist['low'], 0.001)
        self.assertEqual(wd_var._dist['high'], 0.001)

        #Check that action vars are not updated
        model_action_var = self.combined_sampler.action_sampler.get_var('model_action')
        #Since no action indices were provided, the distribution should remain the same
        self.assertEqual(model_action_var.distribution(indices=True), [0, 1, 2])

    def test_trial_with_unknown_variables(self):
        trial_params = {
            'model_action>None': 0,
            'rand_number>model_action': 5,
            'unknown_var>None': 42,
            'lr>None': 0.005,
            'wd>None': 0.001
        }
        trial = self.create_mock_trial(trial_params)

        #Call update and expect that it ignores unknown variables
        self.combined_sampler.update(top_k_trials=[trial], update_state_window=1, device=torch.device('cpu'))

        #Ensure that known variables are updated
        lr_var = self.combined_sampler.get_var('lr')
        self.assertEqual(lr_var._dist['low'], 0.005)
        self.assertEqual(lr_var._dist['high'], 0.005)
        wd_var = self.combined_sampler.get_var('wd')
        self.assertEqual(wd_var._dist['low'], 0.001)
        self.assertEqual(wd_var._dist['high'], 0.001)

        #Check that the ModelAction(and its signature Vars) are updated properly
        model_action_var = self.combined_sampler.action_sampler.get_var('model_action')
        self.assertEqual(model_action_var.distribution(indices=True), [0])
        rand_number_var = model_action_var.get_var(model_action_default,'rand_number')
        self.assertEqual(rand_number_var.distribution(), [6])

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

<ipython-input-36-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('rand_number', 'model_action'), ('activation', 'model_action')]

  warnings.warn(msg)
<ipython-input-36-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('rand_number', 'model_action'), ('activation', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action'), ('activation', 'dls_action'), ('normalize', 'dls_action')]

  warnings.warn(msg)
<ipython-input-36-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('activation', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action'), ('activation', 'dls_action'), ('normalize', 'dls_action')]

 

#Class: Objective

In [91]:
class Objective:
    def __init__(self, trial, combined_sampler: CombinedSampler, device: torch.device, default_dict: dict = {}):
        #Instance of Objective must have an attribute 'execute_objective' set BEFOREHAND
        if not hasattr(self, 'execute_objective'):
            raise RuntimeError("Class Objective must have an attribute 'execute_objective' set before any instance of it can be initialized. This is an internal issue.")

        #Perform general-error checking
        self._validate_inputs(trial=trial, combined_sampler=combined_sampler, default_dict=default_dict)

        #Initialize all the member variables.
        self.trial = trial
        self.default_dict = default_dict
        self.combined_sampler = combined_sampler
        self.var_types = self.combined_sampler.var_types() if self.combined_sampler else set(default_dict.keys())
        self.action_var_types = self.combined_sampler.action_var_types() if self.combined_sampler else {key for key in self.default_dict.keys() if 'action' in key}
        self.sampled_var_types = set()
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def _validate_inputs(self, trial: Trial, combined_sampler: CombinedSampler, default_dict: dict):
        #trial type check:
        if trial is not None and not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' must either be 'None' or an instance in (optuna::Trial, optuna::FrozenTrial) but found: {trial}")

        #combined_sampler type check:
        if combined_sampler is not None and not isinstance(combined_sampler, CombinedSampler):
            raise ValueError(f"Input 'combined_sampler' must either be None or of type CombinedSampler but found: {combined_sampler}")

        #Make sure there's something to be sampled:
        if not default_dict and not combined_sampler:
            raise ValueError(f"Both default_dict and combined_sampler are empty. Nothing to be sampled.")

    def sample(self, var_type: str, default_val = None):
        #No double sampling allowed
        if var_type in self.sampled_var_types:
            raise ValueError(f"Variable '{var_type}' has already been sampled. Variables can only be sampled once per trial.")

        #(1)If var_type in default_dict, sampled_val is retrieved directly from default_dict
        #(2)If not (1) and var_type not in combined_sampler(or no combined_sampler was passed in), sampled_val = default_val
        #(3)If not (1) and (2), then sample from combined_sampler.
        if var_type in self.default_dict.keys():
            sampled_val = self.default_dict[var_type]
        elif self.combined_sampler is None or var_type not in self.var_types:
            sampled_val = default_val
        else: #All False, THEN sampled_val <- sampled value from self.combined_sampler
            sampled_val = self.combined_sampler.sample(var_type, self.trial, default_val)

        #Add var_type to the set of all sampled var_types.
        self.sampled_var_types.add(var_type)

        return sampled_val

#Test: Objective, UserObjective, SampleCallVisitor

In [15]:
from typing import Dict, Any
from torch.utils.data import Dataset, DataLoader

class PolynomialDataset(Dataset):
    def __init__(self, coeffs: torch.tensor, input_min: float, input_max: float, input_size: int):
        self.coeffs = coeffs.clone().detach()
        self.inputs = torch.empty(size=(input_size,)).uniform_(input_min, input_max)
        self.outputs = self._f(X=self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

    def __len__(self):
        return self.inputs.size(0)

    def _f(self, X):
        powers = torch.arange(self.coeffs.size(0)).unsqueeze(0)
        Y = self.inputs.unsqueeze(-1)**powers
        return torch.matmul(Y, self.coeffs).squeeze(-1)

degree = 6
coeffs_min, coeffs_max = -10.0, 10.0
coeffs = torch.empty(size=(degree+1,)).uniform_(coeffs_min, coeffs_max)

train_dset = PolynomialDataset(coeffs=coeffs, input_min=-100.0, input_max=100.0, input_size=8000)
valid_dset = PolynomialDataset(coeffs=coeffs, input_min=-300.12, input_max=500.78, input_size=2000)

class ModelDefault(nn.Module):
    def __init__(self, activation: nn.Module, preprocess: bool, start_neurons: int):
        super().__init__()
        self.preprocess = preprocess
        if self.preprocess:
            start_neurons = max(start_neurons, degree+1)
        self.net = nn.Sequential(
            nn.Linear(in_features=degree+1 if self.preprocess else 1, out_features=start_neurons),
            nn.BatchNorm1d(num_features=start_neurons),
            activation(),
            nn.Linear(in_features=start_neurons, out_features=start_neurons*2),
            nn.BatchNorm1d(num_features=start_neurons*2),
            activation(),
            nn.Linear(in_features=start_neurons*2, out_features=1)
        )

    def _preprocess(self, X):
        N = X.size(0)
        X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
        return X

    def forward(self, X):
        if X.dim() != 1:
            X = X.reshape(-1)

        if self.preprocess:
            X = self._preprocess(X)
        else:
            X = X.unsqueeze(-1)
        return self.net(X)

class ModelDropout(nn.Module):
    def __init__(self, activation: nn.Module, dropout_p: float, preprocess: bool, start_neurons: int):
        super().__init__()
        self.preprocess = preprocess
        if self.preprocess:
            start_neurons = max(start_neurons, degree+1)
        self.net = nn.Sequential(
            nn.Linear(in_features=degree+1 if self.preprocess else 1, out_features=start_neurons),
            nn.BatchNorm1d(num_features=start_neurons),
            activation(),
            nn.Dropout(p=dropout_p),
            nn.Linear(in_features=start_neurons, out_features=start_neurons*2),
            nn.BatchNorm1d(num_features=start_neurons*2),
            activation(),
            nn.Linear(in_features=start_neurons*2, out_features=1)
        )

    def _preprocess(self, X):
        N = X.size(0)
        X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
        return X

    def forward(self, X):
        if X.dim() != 1:
            X = X.reshape(-1)

        if self.preprocess:
            X = self._preprocess(X)
        else:
            X = X.unsqueeze(-1)
        return self.net(X)

def model_action_default(activation: nn.Module, preprocess: bool, start_neurons: int, device: torch.device):
    if start_neurons < 1:
        raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
    return ModelDefault(activation=activation,
                        preprocess=preprocess,
                        start_neurons=start_neurons).to(device)

def model_action_dropout(activation: nn.Module, dropout_p: float, preprocess: bool, start_neurons: int, device: torch.device):
    if start_neurons < 1:
        raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
    if dropout_p < 0.0 or dropout_p >= 1.0:
        raise ValueError(f"0.0 <= dropout_p < 1.0 required but found dropout_p: {dropout_p}")
    return ModelDropout(activation=activation,
                        dropout_p=dropout_p,
                        preprocess=preprocess,
                        start_neurons=start_neurons).to(device)

def dls_action_default(batch_size: int, device=None):
    train_dataloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dset, batch_size=batch_size, shuffle=True)
    dls = DataLoaders(train_dataloader, valid_dataloader)
    return dls

def custom_mse(Y_hat, Y):
    Y_hat = Y_hat.reshape(-1)
    Y = Y.reshape(-1)
    return nn.MSELoss()(Y_hat, Y)

class TestObjective(unittest.TestCase):
    def setUp(self):
        self.INIT_DICT = {
            ('dls_action--action', None): {
                'params': {'choices': [dls_action_default]},
                'sample': 'categorical'
            },
            ('freeze', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('lr_low', None): {
                'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
                'sample': 'float'
            },
            ('lr_high', None): {
                'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('lr', None): {
                'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('wd', None): {
                'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
                'sample': 'categorical'
            },
            ('gradient_clip', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('max_norm', None): {
                'params': {'low': 0.0, 'high': 15.0, 'log': False},
                'sample': 'float'
            },
            ('one_cycle', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('pct_start', None): {
                'params': {'low': 0.10, 'high': 0.95, 'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('n_epoch', None): {
                'params': {'choices': [5, 10, 15]},
                'sample': 'categorical'
            },
            ('model_action--action', None): {
                'params': {'choices': [model_action_default, model_action_dropout]},
                'sample': 'categorical'
            },
            ('activation', 'model_action'): {
                'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
                'sample': 'categorical'
            },
            ('loss_func', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('metric', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('preprocess', 'model_action'): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('start_neurons', 'model_action'): {
                'params': {'choices': [1, 2, 4, 8, 16, 32]},
                'sample': 'categorical'
            },
            ('dropout_p', 'model_action'): {
                'params': {'low': 0.01, 'high': 0.95, 'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('batch_size', 'dls_action'): {
                'params': {'choices': [2, 4, 8, 16, 32, 64]},
                'sample': 'categorical'
            }
        }
        self.cs = UserConfigSpace(init_config=self.INIT_DICT)
        self.trial = optuna.create_study(direction='minimize').ask()
        self.device = torch.device('cpu')

    def test_objective_initialization(self):
        #TEST: Initialziation of ConfigSpace via INIT_DICT
        config_space = UserConfigSpace(init_config=self.INIT_DICT)

        combined_sampler = CombinedSampler(config_space)
        self.assertEqual(
            combined_sampler.var_types(),
            {
            'dls_action', 'freeze', 'lr_low', 'lr_high', 'lr', 'wd',
            'gradient_clip', 'max_norm', 'one_cycle', 'pct_start', 'n_epoch',
            'model_action', 'loss_func', 'metric'
            },
            msg=f"combined_sampler.var_types(): {combined_sampler.var_types()}")
        self.assertEqual(
            combined_sampler.action_var_types(),
            {'model_action', 'dls_action'}
        )
        self.assertEqual(
            combined_sampler.model_action_var_types(),
            {'model_action'}
        )
        self.assertEqual(
            combined_sampler.sig_var_types(),
             {'activation', 'batch_size', 'preprocess', 'dropout_p', 'start_neurons'},
             msg=f"combined_sampler.sig_var_types(): {combined_sampler.sig_var_types()}"
        )
        sampled_dict = dict()
        for key in combined_sampler.var_types():
            sampled_dict[key] = combined_sampler.sample(key, self.trial, default_val=None)
        #This builds shared ownership between the two.
        setattr(Objective, 'execute_objective', self.cs.objective.__func__)
        objective = Objective(trial=self.trial, combined_sampler=combined_sampler, device=self.device)
        self.assertEqual(objective.default_dict, {})

    def test_execute_objective_trial(self):
        #This builds shared ownership between the two.
        setattr(Objective, 'execute_objective', self.cs.objective.__func__)
        combined_sampler = CombinedSampler(INIT_CONFIG=self.cs)
        objective = Objective(trial=self.trial, combined_sampler=combined_sampler, device=self.device, default_dict={})
        objective_loss, model_container = objective.execute_objective()
        self.assertIsInstance(model_container, Learner)
        model = model_container.model
        self.assertIsInstance(model, nn.Module)
        input_sample = torch.randn(size=[64, 1])
        output = model(input_sample)
        self.assertEqual(output.shape, torch.Size([64, 1]))

    #Additional tests can be adapted similarly

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

<ipython-input-8-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('start_neurons', 'model_action'), ('preprocess', 'model_action'), ('activation', 'model_action')]

  warnings.warn(msg)
<ipython-input-8-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('start_neurons', 'model_action'), ('preprocess', 'model_action'), ('activation', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action')]

  warnings.warn(msg)
<ipython-input-8-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('start_neurons', 'model_action'), ('preprocess', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action')]

  warnings.warn(msg

.[I 2024-10-07 01:15:06,486] A new study created in memory with name: no-name-f07f490c-358e-4370-8572-39e0cff29948
.
----------------------------------------------------------------------
Ran 2 tests in 161.312s

OK


#Class: HpoTrainer

In [104]:
class HpoTrainer:
    def __init__(self, combined_sampler: CombinedSampler, n_trials: int, device: torch.device,
                 prev_top_k_trials: list = [], pruner: optuna.pruners = None, timeout: int = 14400, eta: int = 2):
        """
        Class Explanation: HpoTrainer is a TEMPORARY objective for running a single HPO process.
        """
        self.combined_sampler = combined_sampler
        self.device = device
        self.eta = eta
        self.timeout = timeout

        #Enqueue ALL the previous_top_k_trials.
        #NOTE: This allows the optimizer to efficiently search the new space.
        self.prior_trials = prev_top_k_trials

        #Use the user-defined n_trials for new trials (ignore the prior trial count)
        self.n_trials = n_trials

        #Select a pruner if not specified
        choices = (optuna.pruners.HyperbandPruner, optuna.pruners.SuccessiveHalvingPruner)
        self.pruner = random.choice(choices) if pruner is None else pruner

    def run(self):
        print("*****************RUNNING: HPO PROCESS*****************")

        #To get logging info on the HPO process:
        optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

        #Create the study object with pruner
        study = optuna.create_study(
            direction='minimize',
            pruner=self.pruner(reduction_factor=self.eta)
        )

        #Enqueue the prior trials (these will run first)
        for trial in self.prior_trials:
            study.enqueue_trial(trial.params)

        #Run the optimization for the user-defined n_trials (new trials only)
        study.optimize(func=self.hpo_objective, n_trials=self.n_trials, timeout=self.timeout)

        #Return the optimized study
        return study

    def hpo_objective(self, trial):
        #Sanity check
        if not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' MUST be an instance of optuna::Trial OR optuna::FrozenTrial but found trial: {trial}.")

        #Retrieve & return the objective loss, produced by Objective.
        objective_loss, _ = Objective(
            trial=trial,
            combined_sampler=self.combined_sampler,
            device=self.device
        ).execute_objective()

        #Return the objective loss.
        return objective_loss

#Test: HpoTrainer

In [17]:
#Now, adapt the test cases
class TestHpoTrainer(unittest.TestCase):
    def setUp(self):
        self.INIT_DICT = {
            ('dls_action--action', None): {
                'params': {'choices': [dls_action_default]},
                'sample': 'categorical'
            },
            ('freeze', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('lr_low', None): {
                'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
                'sample': 'float'
            },
            ('lr_high', None): {
                'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('lr', None): {
                'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('wd', None): {
                'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
                'sample': 'categorical'
            },
            ('gradient_clip', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('max_norm', None): {
                'params': {'low': 0.0, 'high': 15.0, 'log': False},
                'sample': 'float'
            },
            ('one_cycle', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('pct_start', None): {
                'params': {'low': 0.10, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('n_epoch', None): {
                'params': {'choices': list(range(1,11))},
                'sample': 'categorical'
            },
            ('model_action--action', None): {
                'params': {'choices': [model_action_default, model_action_dropout]},
                'sample': 'categorical'
            },
            ('activation', 'model_action'): {
                'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
                'sample': 'categorical'
            },
            ('loss_func', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('metric', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('preprocess', 'model_action'): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('start_neurons', 'model_action'): {
                'params': {'choices': [1, 2, 4, 8, 16, 32]},
                'sample': 'categorical'
            },
            ('dropout_p', 'model_action'): {
                'params': {'low': 0.01, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('batch_size', 'dls_action'): {
                'params': {'choices': [2, 4, 8, 16, 32, 64]},
                'sample': 'categorical'
            }
        }
        self.cs = UserConfigSpace(init_config=self.INIT_DICT)
        setattr(Objective, 'execute_objective', self.cs.objective.__func__)
        self.device = torch.device('cpu')

    def test_initialization(self):
        combined_sampler = CombinedSampler(INIT_CONFIG=self.cs)

        hpo_trainer = HpoTrainer(
            combined_sampler=combined_sampler,
            n_trials=2,
            device=self.device
        )
        self.assertIsInstance(hpo_trainer, HpoTrainer)
        self.assertIsInstance(hpo_trainer.combined_sampler, CombinedSampler)
        self.assertEqual(hpo_trainer.n_trials, 2)
        self.assertEqual(hpo_trainer.device, self.device)
        self.assertEqual(hpo_trainer.eta, 2)
        self.assertEqual(hpo_trainer.timeout, 14400)

    def test_run(self):
        n_trials = [4, 2, 1]
        top_ks = random.choices(list(range(1, 10)), k=len(n_trials))
        for n, k in zip(n_trials, top_ks):
            combined_sampler = CombinedSampler(INIT_CONFIG=self.cs)

            hpo_trainer = HpoTrainer(
                combined_sampler=combined_sampler,
                n_trials=n,
                device=self.device
            )

            study = hpo_trainer.run()
            prev_trials = study.trials
            top_k = min(len(prev_trials), k)
            top_k_trials = sorted(
                prev_trials,
                key=lambda trial: trial.value if trial.value is not None else float('inf')
            )[:top_k]
            combined_sampler.update(
                top_k_trials=top_k_trials,
                update_state_window=2,
                device=self.device
            )
            #You can add assertions or checks here as needed
            print("****************************************")
            print(combined_sampler)
            print("****************************************")
            print()

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

<ipython-input-8-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('start_neurons', 'model_action'), ('preprocess', 'model_action'), ('activation', 'model_action')]

  warnings.warn(msg)
<ipython-input-8-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('start_neurons', 'model_action'), ('preprocess', 'model_action'), ('activation', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action')]

  warnings.warn(msg)
<ipython-input-8-2e3739082d59>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('start_neurons', 'model_action'), ('preprocess', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action')]

  warnings.warn(msg

*****************RUNNING: HPO PROCESS*****************
A new study created in memory with name: no-name-efc75526-97de-4c57-9c67-dffea4a4396f


[I 2024-10-07 01:15:14,871] Trial 0 finished with value: 2.933249212057006e+32 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 3, 'model_action>None': 0, 'start_neurons>model_action': 3, 'preprocess>model_action': 0, 'activation>model_action': 2, 'loss_func>None': 0, 'metric>None': 0, 'freeze>None': 0, 'gradient_clip>None': 0, 'max_norm>None': 2.4066637230764645, 'one_cycle>None': 1, 'n_epoch>None': 3, 'lr>None': 3.5957951740910273e-06, 'wd>None': 3}. Best is trial 0 with value: 2.933249212057006e+32.


Trial 0 finished with value: 2.933249212057006e+32 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 3, 'model_action>None': 0, 'start_neurons>model_action': 3, 'preprocess>model_action': 0, 'activation>model_action': 2, 'loss_func>None': 0, 'metric>None': 0, 'freeze>None': 0, 'gradient_clip>None': 0, 'max_norm>None': 2.4066637230764645, 'one_cycle>None': 1, 'n_epoch>None': 3, 'lr>None': 3.5957951740910273e-06, 'wd>None': 3}. Best is trial 0 with value: 2.933249212057006e+32.


/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:689: UserWarning: The distribution is specified by [0.01, 0.95] and step=0.05, but the range is not divisible by `step`. It will be replaced by [0.01, 0.91].
  warnings.warn(


...

.[I 2024-10-07 01:23:23,058] A new study created in memory with name: no-name-1e17498a-9d99-4553-9d94-6459bed125f7


****************************************
CombinedSampler:
**********ActionSampler**********
Var of type: model_action, signature of: None
Configuration Space:
'choices' = [<__main__.ModelAction object at 0x7f72bfc9c2b0>]

Var of type: dls_action, signature of: None
Configuration Space:
'choices' = [<__main__.Action object at 0x7f72bfc9fa60>]


**********PrimitiveSampler**********
Var of type: pct_start, signature of: None
Configuration Space:
'low': 0.1, 'high': 0.95, 'log': False, 'step': 0.05

Var of type: wd, signature of: None
Configuration Space:
'choices' = [0.0001]

Var of type: one_cycle, signature of: None
Configuration Space:
'choices' = [False]

Var of type: n_epoch, signature of: None
Configuration Space:
'choices' = [6]

Var of type: lr_low, signature of: None
Configuration Space:
'low': 1e-07, 'high': 0.0009, 'log': True, 'step': unspecified

Var of type: lr, signature of: None
Configuration Space:
'low': 0.02147583062895726, 'high': 0.02147583062895726, 'log': True, 'ste

.[I 2024-10-07 01:24:49,021] A new study created in memory with name: no-name-42b5ecf9-eb5c-43e8-a5b1-d8bd9e40e596


A new study created in memory with name: no-name-42b5ecf9-eb5c-43e8-a5b1-d8bd9e40e596
A new study created in memory with name: no-name-42b5ecf9-eb5c-43e8-a5b1-d8bd9e40e596
A new study created in memory with name: no-name-42b5ecf9-eb5c-43e8-a5b1-d8bd9e40e596


.
----------------------------------------------------------------------
Ran 4 tests in 582.504s

OK


#Class: Auto

In [105]:
class Auto:
    def __init__(self, INIT_CONFIG: ConfigSpace, storage_dir: str = '/content/trial_storage/'):
        if not isinstance(INIT_CONFIG, ConfigSpace):
            raise ValueError(f"Input 'INIT_CONFIG' must be an instance of ConfigSpace but found: {type(INIT_CONFIG)}")

        #Save it as a member.
        self.init_config = INIT_CONFIG
        #Verify that the newly-built ConfigSpace is correct BOTH in terms of syntax AND objective.
        self.init_config.verify_syntax(warn_missing_sig_var_types=False)
        self.init_config.verify_objective(warn_missing_var_types=False)

        self.combined_sampler = CombinedSampler(INIT_CONFIG=self.init_config)

        if self.combined_sampler.is_fixed():
            raise ValueError("Input 'INIT_SOURCE' is completely determined from the beginning. Nothing to optimize for")

        self.prev_top_k_trials = []
        self.storage_dir = storage_dir
        os.makedirs(self.storage_dir, exist_ok=True)
        self._n_study = None

    def optimize(self, n_auto: int, top_k: int, update_state_window: int, device: torch.device):
        #At the start of each optimization request, we set Objective.objective
        setattr(Objective, 'execute_objective', self.init_config.objective.__func__)
        #This verifies that the objective function, which might have been changed by the user,
        try:
            self.init_config.verify_syntax(warn_missing_sig_var_types=False)
            self.init_config.verify_objective(warn_missing_var_types=False)
        except Exception as e:
            raise RuntimeError(f"Following error is raised, possibly due to user's modification to ConfigSpace's objective after Auto has been initialzied: {e}")

        #Now, do the optimization.
        list_n_trials = self._get_list_n_trials(n_auto=n_auto)
        print("*****************************************************************")
        print(f"n_auto: {n_auto}, list_n_trials: {list_n_trials}")
        print()

        for idx, n_trials in enumerate(list_n_trials):
            hpo_pointer = HpoTrainer(
                combined_sampler=self.combined_sampler,
                n_trials=n_trials,
                device=device,
                prev_top_k_trials=self.prev_top_k_trials
            )
            optimized_study = hpo_pointer.run()
            self._update(
                idx=idx,
                optimized_study=optimized_study,
                top_k=top_k,
                update_state_window=update_state_window,
                device=device
            )
            self._n_study = len(list_n_trials)

    def num_study(self):
        return self._n_study

    def get_study(self, idx: int):
        if self._n_study is None:
            raise SyntaxError(".visualization_trials is requested w/o saved data. Call .optimize() first and once it's finished, request visualization again.")
        if idx < 0 or idx >= self._n_study:
            raise ValueError(f"Input 'idx':{idx} must satisfy 0 <= idx < {self._n_study} but failed.")

        return self._load_study_from_pickle(idx)

    def _get_list_n_trials(self, n_auto: int):
        """
        ACCEPTS:
        n_auto := Positive integer power of 2. IF n_auto <= 1, THEN n_auto <- 2
                                               IF n_auto > 1 but NOT a power of 2, THEN n_auto <- (nearest power of 2) - 1.
        RETURNS:
        Exponential distribution of [2^m, 2^(m-1), ..., 2^1, 2^0] + (n_auto==power of 2)*[2^0] trials returned.
        """
        n_auto = max(n_auto, 2)
        pow2 = n_auto & (n_auto-1) == 0

        output = [1<<e for e in range(math.ceil(math.log2(n_auto)) - 1, -1, -1)]

        if pow2:
            output.append(1)

        return output

    def _update(self, idx: int, optimized_study: optuna.study, top_k: int, update_state_window: int, device: torch.device):
        prev_trials = optimized_study.trials
        top_k = min(top_k, len(prev_trials))
        update_state_window = min(update_state_window, top_k)

        top_k_trials = sorted(prev_trials, key=lambda trial: trial.value if trial.value is not None else float('inf'))[:top_k]

        #Save the whole study, not just top_k_trials
        self._save_study_to_pickle(idx, optimized_study)
        self.prev_top_k_trials = top_k_trials

        self.combined_sampler.update(
            top_k_trials=top_k_trials,
            update_state_window=update_state_window,
            device=device
        )

    def _save_study_to_pickle(self, idx: int, study: optuna.study):
        save_path = os.path.join(self.storage_dir, f'study_{idx}.pkl')
        with gzip.open(save_path, 'wb') as f:
            pickle.dump(study, f)
        print(f"Saved study to: {save_path}")

    def _load_study_from_pickle(self, idx: int):
        load_path = os.path.join(self.storage_dir, f'study_{idx}.pkl')
        if not os.path.exists(load_path):
            raise FileNotFoundError(f"No study found at: {load_path}")

        with gzip.open(load_path, 'rb') as f:
            study = pickle.load(f)
        print(f"Loaded study from: {load_path}")
        return study

#Test: Auto

In [106]:
if __name__ == '__main__':
    from torch.utils.data import Dataset

    class PolynomialDataset(Dataset):
        def __init__(self, coeffs: torch.tensor, input_min: float, input_max: float, input_size: int):
            #coeffs.shape == [degree+1, 1]
            self.coeffs = coeffs.clone().detach()
            #inputs.shape == [inputs]
            self.inputs = torch.empty(size=(input_size,)).uniform_(input_min, input_max)
            #Preprocess the output and retrieve it upon request.
            self.outputs = self._f(X=self.inputs)

        def __getitem__(self, idx):
            return self.inputs[idx], self.outputs[idx]

        def __len__(self):
            if self.inputs.size(0) != self.outputs.size(0):
                return ValueError(f"len(inputs): {self.inputs.size(0)} != len(outputs): {self.outputs.size(0)}")
            return self.inputs.size(0)

        def _f(self, X):
            #power.shape == [1, degree+1]
            powers = torch.arange(self.coeffs.size(0)).unsqueeze(0)
            Y = self.inputs.unsqueeze(-1)**powers
            return torch.matmul(Y, self.coeffs).squeeze(-1)

    degree = 6
    coeffs_min, coeffs_max = -10.0, 10.0
    coeffs = torch.empty(size=(degree+1,)).uniform_(coeffs_min, coeffs_max)

    for c in coeffs:
        print(f"c: {c}")

    train_dset = PolynomialDataset(coeffs=coeffs, input_min=-100.0, input_max=100.0, input_size=8000)
    valid_dset = PolynomialDataset(coeffs=coeffs, input_min=-300.12, input_max=500.78, input_size=2000)

    #Test:
    print(f"Number of elements in Dataset; train_dset: {len(train_dset)}, valid_dset: {len(valid_dset)}")
    print()

    print(f"train first input: {train_dset[0][0]}, train first output: {train_dset[0][1]}")
    print()

    print(f"valid first input: {valid_dset[0][0]}, valid first output: {valid_dset[0][1]}")
    print()

    """
    Action functions:
    """
    class ModelDefault(nn.Module):
        def __init__(self, activation: nn.Module, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess: start_neurons = max(start_neurons,degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1,
                        out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Linear(in_features=start_neurons,out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2,out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            #X <- [64,1]x[1,7] == [64,7], where each index (i,j) yields X[i]**j.
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            if X.size() != torch.Size([N,degree+1]):
                raise ValueError(f"X.size() MUST be torch.Size([{N},{degree+1}]) but found X.size()=={X.size()}")
            return X

        def forward(self, X):
            if X.dim() != 1:
                raise ValueError(f"X.dim() MUST be 1 but found X.dim()=={X.dim()}")

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    class ModelDropout(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: int, preprocess: int, start_neurons: Int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess: start_neurons = max(start_neurons,degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1,
                        out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=start_neurons,out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2,out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            #X <- [64,1]x[1,7] == [64,7], where each index (i,j) yields X[i]**j.
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            if X.size() != torch.Size([N,degree+1]):
                raise ValueError(f"X.size() MUST be torch.Size([{N},{degree+1}]) but found X.size()=={X.size()}")
            return X

        def forward(self, X):
            if X.dim() != 1:
                raise ValueError(f"X.dim() MUST be 1 but found X.dim()=={X.dim()}")

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    def action_default(activation: nn.Module, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        return ModelDefault(activation=activation,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    def action_dropout(activation: nn.Module, dropout_p: int, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        if dropout_p < 0.0 or dropout_p >= 1.0:
            raise ValueError(f"0.0 <= dropout_p < 1.0 required but found dropout_p: {dropout_p}")
        return ModelDropout(activation=activation,
                            dropout_p=dropout_p,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    #Define DataLoaders and training process
    def get_dls(batch_size: int):
        train_dataloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        valid_dataloader = DataLoader(valid_dset, batch_size=batch_size, shuffle=True)

        dls = DataLoaders(train_dataloader, valid_dataloader)
        return dls

    # Now you can use this DataLoaders object with Learner
    # Example usage:
    train_dls, valid_dls = get_dls(batch_size=64)

    print(f"len(train_dls): {len(train_dls)}")
    print(f"len(valid_dls): {len(valid_dls)}")
    print()

    print("*****CHECKING data integrity of returned 'dls'******")
    for blob in iter(train_dls):
        if not isinstance(blob, tuple):
            raise ValueError(f"Sampled 'blob' from 'iter(train_dls)' MUST have a type 'tuple' but found: {type(blob)}")
        inputs, outputs = blob
        if inputs.size() != outputs.size():
            raise ValueError(f"train_dls contains (inputs,outputs) where inputs.size(): {inputs.size()} != outputs.size(): {outputs.size()}")

    for blob in iter(valid_dls):
        if not isinstance(blob, tuple):
            raise ValueError(f"Sampled 'blob' from 'iter(valid_dls)' MUST have a type 'tuple' but found: {type(blob)}")
        inputs, outputs = blob
        if inputs.size() != outputs.size():
            raise ValueError(f"valid_dls contains (inputs,outputs) where inputs.size(): {inputs.size()} != outputs.size(): {outputs.size()}")

    inputs, _ = next(iter(train_dls))
    print(inputs.size())
    inputs, _ = next(iter(valid_dls))
    print(inputs.size())

    model_default = ModelDefault(activation=nn.ReLU, preprocess=False, start_neurons=7)
    model_dropout = ModelDropout(activation=nn.ReLU, dropout_p=0.5, preprocess=False, start_neurons=7)

    print("*********************model_default*********************")
    print(model_default)
    print()
    print("*********************model_dropout*********************")
    print(model_dropout)
    print()

    model_container = Learner(
        dls=get_dls(batch_size=64),
        model=ModelDefault(activation=nn.ReLU, preprocess=False, start_neurons=7),
        loss_func=custom_mse,
        metrics=[custom_mse]
    )
    model_container.fit_one_cycle(n_epoch=10, lr_max=slice(1e-6,1e-3), pct_start=0.3, wd=0.01)

c: -8.256911277770996
c: 2.7582085132598877
c: 8.531203269958496
c: -5.975925922393799
c: -7.2286248207092285
c: 8.258957862854004
c: -0.5422282218933105
Number of elements in Dataset; train_dset: 8000, valid_dset: 2000

train first input: 32.70071792602539, train first output: -362659904.0

valid first input: 483.7815246582031, valid first output: -6733022124572672.0

len(train_dls): 125
len(valid_dls): 32

*****CHECKING data integrity of returned 'dls'******
torch.Size([64])
torch.Size([64])
*********************model_default*********************
ModelDefault(
  (net): Sequential(
    (0): Linear(in_features=1, out_features=7, bias=True)
    (1): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=7, out_features=14, bias=True)
    (4): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=14, out_features=1, bias=True)
  )
)

******************

epoch,train_loss,valid_loss,custom_mse,time
0,23401500800951141793792.000000,3173531826257249212454387318784.000000,3173531826257249212454387318784.000000,00:00
1,24074018829706750590976.000000,3173532430720159019768974671872.000000,3173532430720159019768974671872.000000,00:00
2,23002779111541522300928.000000,3173532128488704116111680995328.000000,3173532128488704116111680995328.000000,00:00
3,23810074114545634246656.000000,3173532128488704116111680995328.000000,3173532128488704116111680995328.000000,00:00
4,24002530941021685022720.000000,3173532430720159019768974671872.000000,3173532430720159019768974671872.000000,00:00
5,23257342828678825902080.000000,3173532128488704116111680995328.000000,3173532128488704116111680995328.000000,00:00
6,23449169151207044808704.000000,3173532128488704116111680995328.000000,3173532128488704116111680995328.000000,00:00
7,23670347684306650923008.000000,3173531826257249212454387318784.000000,3173531826257249212454387318784.000000,00:00
8,23844515392695950114816.000000,3173531826257249212454387318784.000000,3173531826257249212454387318784.000000,00:00
9,24109068093806761476096.000000,3173532430720159019768974671872.000000,3173532430720159019768974671872.000000,00:00


In [107]:
if __name__ == '__main__':
    def custom_mse(Y_hat, Y):
        Y_hat = Y_hat.reshape(-1)
        Y = Y.reshape(-1)
        if Y_hat.size() != Y.size():
            raise ValueError(f"Y_hat.size(): {Y_hat.size()} != Y.size(): {Y.size()}")
        return nn.MSELoss()(Y_hat, Y)

    INIT_DICT = {
        ('dls_action--action', None): {
            'params': {'choices': [get_dls]},
            'sample': 'categorical'
        },
        ('freeze', None) : {
            'params': {'choices': [True, False]},
            'sample': 'categorical'
        },
        ('lr_low', None) : {
            'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
            'sample': 'float'
        },
        ('lr_high', None) : {
            'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
            'sample': 'float'
        },
        ('lr', None) : {
            'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
            'sample': 'float'
        },
        ('wd', None) : {
            'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
            'sample': 'categorical'
        },
        ('gradient_clip', None) : {
            'params': {'choices': [True, False]},
            'sample': 'categorical'
        },
        ('max_norm', None) : {
            'params': {'low': 0.0, 'high': 15.0, 'log': False},
            'sample': 'float'
        },
        ('one_cycle', None) : {
            'params': {'choices': [True, False]},
            'sample': 'categorical'
        },
        ('pct_start', None) : {
            'params': {'low': 0.10, 'high': 0.95,  'log': False, 'step': 0.05},
            'sample': 'float'
        },
        ('n_epoch', None) : {
            'params': {'choices': [5, 10, 15]},
            'sample': 'categorical'
        },
        ('model_action--action', None) : {
            'params': {'choices': [action_default, action_dropout]},
            'sample': 'categorical'
        },
        ('activation', 'model_action') : {
            'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
            'sample': 'categorical'
        },
        ('loss_func', None) : {
            'params': {'choices': [custom_mse]},
            'sample': 'categorical'
        },
        ('metric', None) : {
            'params': {'choices': [custom_mse]},
            'sample': 'categorical'
        },
        ('preprocess', 'model_action') : {
            'params': {'choices': [True, False]},
            'sample': 'categorical'
        },
        ('start_neurons', 'model_action') : {
            'params': {'choices': [1, 2, 4, 8, 16, 32,]},
            'sample': 'categorical'
        },
        ('dropout_p', 'model_action') : {
            'params': {'low': 0.05, 'high': 0.95,  'log': False, 'step': 0.05},
            'sample': 'float'
        },
        ('batch_size', 'dls_action') : {
            'params': {'choices': [32, 64, 128]},
            'sample': 'categorical'
        }
    }

    init_config = UserConfigSpace(INIT_DICT)
    auto = Auto(init_config)

<ipython-input-102-48ea770699ec>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('start_neurons', 'model_action'), ('preprocess', 'model_action'), ('activation', 'model_action')]

  warnings.warn(msg)
<ipython-input-102-48ea770699ec>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('start_neurons', 'model_action'), ('preprocess', 'model_action'), ('activation', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action')]

  warnings.warn(msg)
<ipython-input-102-48ea770699ec>:193: UserWarning: Action hyperparameter 'model_action' requires additional hyperparameters: [('dropout_p', 'model_action'), ('start_neurons', 'model_action'), ('preprocess', 'model_action')]
Action hyperparameter 'dls_action' requires additional hyperparameters: [('batch_size', 'dls_action')]

  warnings.wa

#FINAL_TEST

In [108]:
auto.optimize(
    n_auto=10,
    top_k=2,
    update_state_window=2,
    device=torch.device('cpu')
)

#Print out the optimized combined_sampler.
print()
print("*****************************************************************")
print()
print(auto.combined_sampler)
print()
print("*****************************************************************")
print()

#TEST: Optuna's visualization capabilities
print(auto.num_study())
study = auto.get_study(2)
#One can now perform all the optuna visualizations as one sees fit
optuna.visualization.plot_optimization_history(study)

<ipython-input-102-48ea770699ec>:142: UserWarning: Following calls in '.objective' are referring to non-existing var_type(s) and will simply return respective default_val(s): ['self.sample(min_delta,0.1)', 'self.sample(patience,15)'].
  warnings.warn(f"Following calls in '.objective' are referring to non-existing var_type(s) and will simply return respective default_val(s): {formatted}.")
[I 2024-10-07 04:20:03,124] A new study created in memory with name: no-name-53bb76d4-4842-4695-818e-1be2107cbffd


*****************************************************************
n_auto: 10, list_n_trials: [8, 4, 2, 1]

*****************RUNNING: HPO PROCESS*****************
A new study created in memory with name: no-name-53bb76d4-4842-4695-818e-1be2107cbffd
A new study created in memory with name: no-name-53bb76d4-4842-4695-818e-1be2107cbffd
A new study created in memory with name: no-name-53bb76d4-4842-4695-818e-1be2107cbffd
A new study created in memory with name: no-name-53bb76d4-4842-4695-818e-1be2107cbffd


[I 2024-10-07 04:20:09,288] Trial 0 finished with value: 3.173531826257249e+30 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 2, 'model_action>None': 1, 'dropout_p>model_action': 0.8, 'start_neurons>model_action': 1, 'preprocess>model_action': 0, 'activation>model_action': 0, 'loss_func>None': 0, 'metric>None': 0, 'freeze>None': 0, 'gradient_clip>None': 1, 'one_cycle>None': 1, 'n_epoch>None': 2, 'lr>None': 0.004740235218431013, 'wd>None': 1}. Best is trial 0 with value: 3.173531826257249e+30.


Trial 0 finished with value: 3.173531826257249e+30 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 2, 'model_action>None': 1, 'dropout_p>model_action': 0.8, 'start_neurons>model_action': 1, 'preprocess>model_action': 0, 'activation>model_action': 0, 'loss_func>None': 0, 'metric>None': 0, 'freeze>None': 0, 'gradient_clip>None': 1, 'one_cycle>None': 1, 'n_epoch>None': 2, 'lr>None': 0.004740235218431013, 'wd>None': 1}. Best is trial 0 with value: 3.173531826257249e+30.
Trial 0 finished with value: 3.173531826257249e+30 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 2, 'model_action>None': 1, 'dropout_p>model_action': 0.8, 'start_neurons>model_action': 1, 'preprocess>model_action': 0, 'activation>model_action': 0, 'loss_func>None': 0, 'metric>None': 0, 'freeze>None': 0, 'gradient_clip>None': 1, 'one_cycle>None': 1, 'n_epoch>None': 2, 'lr>None': 0.004740235218431013, 'wd>None': 1}. Best is trial 0 with value: 3.173531826257249e+30.
Trial 0 finished with valu

[I 2024-10-07 04:20:12,679] Trial 1 finished with value: 3.173532128488704e+30 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 1, 'model_action>None': 0, 'start_neurons>model_action': 5, 'preprocess>model_action': 0, 'activation>model_action': 2, 'loss_func>None': 0, 'metric>None': 0, 'freeze>None': 1, 'gradient_clip>None': 1, 'one_cycle>None': 1, 'n_epoch>None': 0, 'lr>None': 0.0024608937892963054, 'wd>None': 0}. Best is trial 0 with value: 3.173531826257249e+30.


...

[I 2024-10-07 04:21:38,888] A new study created in memory with name: no-name-f790750f-cc95-475d-b8d2-41345da64cf5


*****************RUNNING: HPO PROCESS*****************
A new study created in memory with name: no-name-f790750f-cc95-475d-b8d2-41345da64cf5
A new study created in memory with name: no-name-f790750f-cc95-475d-b8d2-41345da64cf5
A new study created in memory with name: no-name-f790750f-cc95-475d-b8d2-41345da64cf5
A new study created in memory with name: no-name-f790750f-cc95-475d-b8d2-41345da64cf5
A new study created in memory with name: no-name-f790750f-cc95-475d-b8d2-41345da64cf5
A new study created in memory with name: no-name-f790750f-cc95-475d-b8d2-41345da64cf5
A new study created in memory with name: no-name-f790750f-cc95-475d-b8d2-41345da64cf5


[I 2024-10-07 04:21:45,362] Trial 0 finished with value: 3.173531826257249e+30 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 2, 'model_action>None': 1, 'loss_func>None': 0, 'metric>None': 0, 'gradient_clip>None': 1, 'one_cycle>None': 0, 'lr_low>None': 0.0005876567979976106, 'lr_high>None': 0.07571540414942228, 'n_epoch>None': 2, 'wd>None': 3, 'pct_start>None': 0.2}. Best is trial 0 with value: 3.173531826257249e+30.


Trial 0 finished with value: 3.173531826257249e+30 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 2, 'model_action>None': 1, 'loss_func>None': 0, 'metric>None': 0, 'gradient_clip>None': 1, 'one_cycle>None': 0, 'lr_low>None': 0.0005876567979976106, 'lr_high>None': 0.07571540414942228, 'n_epoch>None': 2, 'wd>None': 3, 'pct_start>None': 0.2}. Best is trial 0 with value: 3.173531826257249e+30.
Trial 0 finished with value: 3.173531826257249e+30 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 2, 'model_action>None': 1, 'loss_func>None': 0, 'metric>None': 0, 'gradient_clip>None': 1, 'one_cycle>None': 0, 'lr_low>None': 0.0005876567979976106, 'lr_high>None': 0.07571540414942228, 'n_epoch>None': 2, 'wd>None': 3, 'pct_start>None': 0.2}. Best is trial 0 with value: 3.173531826257249e+30.
Trial 0 finished with value: 3.173531826257249e+30 and parameters: {'dls_action>None': 0, 'batch_size>dls_action': 2, 'model_action>None': 1, 'loss_func>None': 0, 'metric>None': 0,


*****************************************************************

CombinedSampler:
**********ActionSampler**********
Var of type: model_action, signature of: None
Configuration Space:
'choices' = [<__main__.ModelAction object at 0x7f72bfc9cb50>]

Var of type: dls_action, signature of: None
Configuration Space:
'choices' = [<__main__.Action object at 0x7f72bfc9d6c0>]


**********PrimitiveSampler**********
Var of type: pct_start, signature of: None
Configuration Space:
'low': 0.2, 'high': 0.2, 'log': False, 'step': 0.05

Var of type: wd, signature of: None
Configuration Space:
'choices' = [0.1]

Var of type: one_cycle, signature of: None
Configuration Space:
'choices' = [True]

Var of type: n_epoch, signature of: None
Configuration Space:
'choices' = [15]

Var of type: lr_low, signature of: None
Configuration Space:
'low': 0.0005876567979976106, 'high': 0.0005876567979976106, 'log': True, 'step': unspecified

Var of type: lr, signature of: None
Configuration Space:
'low': 0.00474023521